# 这里是NPCNN的数据预处理部分

In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
import sys, re
import os
import csv
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import f1_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch.nn as nn
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time
from torch.nn import CrossEntropyLoss
import datetime
import os
import json
from unqlite import UnQLite
import string
import re
import inflection
import nltk
from nltk.stem.porter import PorterStemmer
from assets import stop_words, java_keywords
from parsers import Parser
import pandas as pd
from collections import namedtuple
# dataTest=pd.DataFrame()
# dataTest['id']=[1,2,3]
# dataTest['name']=['hdj','gzh','cxl']
# dataTest.to_pickle('/data/hdj/cross_project_trans/report/aspectj_test_middle/student.pkl')


In [178]:
bug_report_files_dataframe=train_all_dataframe.loc['b067e67']
pos=bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix']==1.0][['used_in_fix']]
neg=bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix']==0.0][['used_in_fix']]

In [179]:
pos.index.name='fid'
# bug_report_files_dataframe
pos
# bug_report_files_dataframe

,used_in_fix
fid,
94065bf02a63710434075209886f06f9464ed3a1,1.0
8d7063afbecfe86cb9ec5be3e059633a488a1c4f,1.0


In [180]:
neg.shape
pos=pos.reset_index()
pos['bid']='a'
pos

,fid,used_in_fix,bid
0,94065bf02a63710434075209886f06f9464ed3a1,1.0,a
1,8d7063afbecfe86cb9ec5be3e059633a488a1c4f,1.0,a


# 生成train pos neg数据

In [65]:
# train_all_dataframe=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_training_fold_'+str(0)+'_raw')
# print(train_all_dataframe.shape)
# for i in range(1,7):
#     train=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_training_fold_'+str(i)+'_raw')
#     print(train.shape)
#     train_all_dataframe=pd.concat([train_all_dataframe,train])
# print(train_all_dataframe.shape)
all_results_df_before.head()

used_in_fix        f1  \
bid     fid                                                               
a0c9251 aa245982fa51518c22c1817fd56ac88b4b921f80          0.0  0.030239   
        38b760ae592520a29eecbc29ac4b6665c9e9bf61          0.0  0.010100   
        ebde7ae63aa647dc84eb1c29f1fa2133b4d1e705          0.0  0.026023   
        0b6789bd6c4f2ad5b82d0e4800e6c3924bc45589          0.0  0.026685   
        bc6b590ab782fc46728d0561c084f3c5fa843a68          0.0  0.040819   

                                                    result  
bid     fid                                                 
a0c9251 aa245982fa51518c22c1817fd56ac88b4b921f80  0.588582  
        38b760ae592520a29eecbc29ac4b6665c9e9bf61  0.556395  
        ebde7ae63aa647dc84eb1c29f1fa2133b4d1e705  0.549675  
        0b6789bd6c4f2ad5b82d0e4800e6c3924bc45589  0.550011  
        bc6b590ab782fc46728d0561c084f3c5fa843a68  0.600155

,used_in_fix,f1,result
fid,,,
b5f5e1a2070ba15a8d5a8cd955de857225e0815b,0.0,0.185988,0.940310
6f547898bd6d011af1e621a2c002f10c39856cd4,0.0,0.159427,0.928099
b759c9f92786cc5b07329d303f2bfe37239d0db3,0.0,0.159427,0.926931
e7a5e5ff9e842566fce2b344076eda8486db4c1a,1.0,0.159427,0.926125
638a1fbe3d9de7b7623c1fac1260408526230971,0.0,0.233812,0.924647


,bid,fid,used_in_fix
0,1757498,b5f5e1a2070ba15a8d5a8cd955de857225e0815b,0
1,1757498,6f547898bd6d011af1e621a2c002f10c39856cd4,0
2,1757498,b759c9f92786cc5b07329d303f2bfe37239d0db3,0
3,1757498,e7a5e5ff9e842566fce2b344076eda8486db4c1a,1
4,1757498,638a1fbe3d9de7b7623c1fac1260408526230971,0


,bid,fid,used_in_fix
3,1757498,e7a5e5ff9e842566fce2b344076eda8486db4c1a,1
185,1757498,146f43ae14ef2b9d67dd1ebc1e89301d7da4f0d5,1


In [69]:
neg.head()

,bid,fid,used_in_fix
0,1757498,b5f5e1a2070ba15a8d5a8cd955de857225e0815b,0
1,1757498,6f547898bd6d011af1e621a2c002f10c39856cd4,0
2,1757498,b759c9f92786cc5b07329d303f2bfe37239d0db3,0
4,1757498,638a1fbe3d9de7b7623c1fac1260408526230971,0
5,1757498,b6981723eb755a4f1b4e7883c027f9891d227341,0


In [74]:
train_set=train_all_dataframe.index.get_level_values(0).unique()

In [75]:
train_set & test_set

Index(['a0c9251', '1c62052', '6fac09a', 'b4a3cd3', '8380cc1', '981f2e3',
       'f3c9326', '36baafc', '71b4d72', 'f664172',
       ...
       '2596dc6', '6694f7a', '2333840', '00d5736', '805981b', '9dd9bf7',
       '149e29d', 'e0bfce0', '60838e2', 'f248d27'],
      dtype='object', length=3000)

In [73]:
test_set=all_results_df_before.index.get_level_values(0).unique()

In [79]:
len(train_set),len(test_set)
# i=0
# for key in train_set:
#     if key not in test_set:
#         print(key)
#         i+=1
# i

(3500, 3645)

In [83]:
all_results_df_before=pd.read_pickle('/data/hdj/tracking_buggy_files/joblib_memmap_'+project+'/all_results_df_before.pickle')
pos_train=[]
neg_train=[]
for i,(bug_report, bug_report_files_dataframe) in enumerate(train_all_dataframe.groupby(level=0, sort=False)):
    try:
        result=all_results_df_before.loc[bug_report]
        sorted_df = result.sort_values(ascending=False, by=['result'])
        sorted_df.head()
        sorted_df=sorted_df.head(300)
        sorted_df=sorted_df.reset_index()
        sorted_df['bid']=bug_report
        sorted_df = sorted_df[['bid','fid','used_in_fix']]
        sorted_df[["used_in_fix"]] = sorted_df[["used_in_fix"]].astype(int)
        sorted_df.head()
        pos=sorted_df[sorted_df['used_in_fix']==1.0]
        pos_size=pos.shape[0]
        neg=sorted_df[sorted_df['used_in_fix']==0.0].head(3*pos_size)
        pos_train.append(pos)
        neg_train.append(neg)
        pos.head()
    except:
        continue
pos_dataframe=pd.concat(pos_train,axis=0)
neg_dataframe=pd.concat(neg_train,axis=0)
pos_train_path=os.path.join(outpath,project+'_80_training_pos.csv')
neg_train_path=os.path.join(outpath,project+'_80_training_neg.csv')
pos_dataframe.to_csv(pos_train_path,index=False)
neg_dataframe.to_csv(neg_train_path,index=False)

In [39]:
#换种简单数据集，正例和一半高，一半低的负例
project='swt'
all_results_df_before=pd.read_pickle('/data/hdj/tracking_buggy_files/joblib_memmap_'+project+'/all_results_df_before.pickle')
all_results_df_before.head()
outpath='/data/hdj/cross_project_trans/report/bert/'
test_all_dataframe=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_testing_fold_'+str(7))
test_all_dataframe.head()
outpath='/data/hdj/cross_project_trans/report/bert/'
for i,(bug_report, bug_report_files_dataframe) in enumerate(test_all_dataframe.groupby(level=0, sort=False)):
    result=all_results_df_before.loc[bug_report]
    sorted_df = result.sort_values(ascending=False, by=['result'])
    sorted_df_head=sorted_df.head(300)
    sorted_df_head=sorted_df_head.reset_index()
    sorted_df_tail=sorted_df.tail(300)
    sorted_df_tail=sorted_df_tail.reset_index()
    pos=sorted_df_head[sorted_df_head['used_in_fix']==1.0]
    neg=sorted_df_tail[sorted_df_tail['used_in_fix']==0.0].head(300-pos.shape[0])
    test=pd.concat([pos,neg],axis=0)
    test['bid']=bug_report
    test = test[['bid','fid','used_in_fix']]
    test[["used_in_fix"]] = test[["used_in_fix"]].astype(int)
    writePath=os.path.join(outpath,project+'_test','test'+str(i)+'.csv')
    test.to_csv(writePath,index=False)

In [32]:
test=pd.concat([pos,neg],axis=0)
test.head()

,fid,used_in_fix,f1,result
0,ddab53ae4e1bc70e98dbf26260e71628bd49c373,1.0,0.504215,1.279086
0,b739306765c98742f349668310963884ebb38b4f,0.0,0.067268,0.674424
1,606bf4e86cd84b680fbc7dd872f364c5c9d8398f,0.0,0.087014,0.674344
2,dd75807a917eef64b619b90c5493549c4ddfff26,0.0,0.087723,0.674329
3,9ee0c1d55b4aa76a35150babfbda99a17e468a58,0.0,0.087115,0.674315


In [29]:

neg.shape
neg.head()

,fid,used_in_fix,f1,result
0,b739306765c98742f349668310963884ebb38b4f,0.0,0.067268,0.674424
1,606bf4e86cd84b680fbc7dd872f364c5c9d8398f,0.0,0.087014,0.674344
2,dd75807a917eef64b619b90c5493549c4ddfff26,0.0,0.087723,0.674329
3,9ee0c1d55b4aa76a35150babfbda99a17e468a58,0.0,0.087115,0.674315
4,7387bb43e5dd637fa5d6aa806171cc1376df63f8,0.0,0.090022,0.674296


In [86]:
# neg_dataframe

In [85]:
# pos_dataframe

,bid,fid,used_in_fix
3,a0c9251,e7a5e5ff9e842566fce2b344076eda8486db4c1a,1
185,a0c9251,146f43ae14ef2b9d67dd1ebc1e89301d7da4f0d5,1
1,1c62052,e7dd32ae2b70ff5babc76f6de822712139134480,1
28,1c62052,43728ad151d9f048e1794736411d2ef330ce2b31,1
3,6fac09a,f10b4f2d0f2c9a1986aed279d6c24579c570d2f4,1
...,...,...,...
5,e0bfce0,afac649fe074e4e154a926a6fde59bb46226929d,1
2,60838e2,7091e82439940977000c6e0aff2668659367ab34,1
83,60838e2,80334199af4c21ea2a3a4c7ee175c23d3192851e,1
4,f248d27,84620e4c9b0022e46ae547923991f3d8b19f81fc,1


In [63]:
pos_train=[]
neg_train=[]
train_all_dataframe=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_training_fold_'+str(0)+'_raw')
for i in range(1,7):
    train=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_training_fold_'+str(i)+'_raw')
    print(train.shape)
    train_all_dataframe=pd.concat([train_all_dataframe,train])

# train_all_dataframe.tail()
for i,(bug_report, bug_report_files_dataframe) in enumerate(train_all_dataframe.groupby(level=0, sort=False)):
    pos=bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix']==1.0][['used_in_fix']]
    neg=bug_report_files_dataframe[bug_report_files_dataframe['used_in_fix']==0.0][['used_in_fix']]
    neg_size=pos.shape[0]
    neg=neg.head(2*neg_size)
    pos[["used_in_fix"]] = pos[["used_in_fix"]].astype(int)
    neg[["used_in_fix"]] = neg[["used_in_fix"]].astype(int)
#     print(bug_report_files_dataframe)
    pos=pos.reset_index()
    neg=neg.reset_index()
#     print(pos)
    pos_train.append(pos)
    neg_train.append(neg)
#     if i==2:
#         break
pos_dataframe=pd.concat(pos_train,axis=0)
neg_dataframe=pd.concat(neg_train,axis=0)
pos_dataframe.rename(columns={'level_0':'bid','level_1':'fid'},inplace=True)
neg_dataframe.rename(columns={'level_0':'bid','level_1':'fid'},inplace=True)
pos_train_path=os.path.join(outpath,project+'_80_training_pos.csv')
neg_train_path=os.path.join(outpath,project+'_80_training_neg.csv')
pos_dataframe.to_csv(pos_train_path,index=False)
neg_dataframe.to_csv(neg_train_path,index=False)

(100907, 20)
(101025, 20)
(100882, 20)
(100915, 20)
(100980, 20)
(101391, 20)


KeyboardInterrupt: 

# 生成train pos neg数据完成

In [201]:
pos_dataframe.rename(columns={'level_0':'bid','level_1':'fid'},inplace=True)
neg_dataframe.rename(columns={'level_0':'bid','level_1':'fid'},inplace=True)
pos_dataframe.head()

,bid,fid,used_in_fix
0,1839d9b,8640d87eebe944043533a62885e33b04d5475d53,1
0,5ce168a,77adc4b41a8fc3eed6d7c62292be018cda642669,1
1,5ce168a,6cbed3de32cbfb7970ff47cd2ae634138c2e5d8a,1
2,5ce168a,f61551cda8113971da3ea148a77d208ff4b89c5a,1
0,83e8870,4ad29aee46529969cf18949646ed741ae6703f10,1


In [203]:
neg_dataframe.head()
pos_dataframe.shape,neg_dataframe.shape

((10, 3), (600, 3))

# 生成test的数据

In [48]:
all_results_df_before=pd.read_pickle('/data/hdj/tracking_buggy_files/joblib_memmap_'+project+'/all_results_df_before.pickle')
all_results_df_before.head()
project='swt'
test_all_dataframe=pd.read_pickle('/data/hdj/tracking_buggy_files/'+project+'/'+project+'_normalized_testing_fold_'+str(7))
test_all_dataframe.head()
outpath='/data/hdj/cross_project_trans/report/bert/'
for i,(bug_report, bug_report_files_dataframe) in enumerate(test_all_dataframe.groupby(level=0, sort=False)):
    
    result=all_results_df_before.loc[bug_report]
    sorted_df = result.sort_values(ascending=False, by=['result'])
    sorted_df=sorted_df.head(300)
    sorted_df=sorted_df.reset_index()
    sorted_df.head()
    writePath=os.path.join(outpath,project+'_test','test'+str(i)+'.csv')
#     print(sum(bug_report_files_dataframe['used_in_fix']))
    sorted_df=sorted_df[['fid','used_in_fix']]
    sorted_df['bid']=bug_report
    sorted_df = sorted_df[['bid','fid','used_in_fix']]
    sorted_df[["used_in_fix"]] = sorted_df[["used_in_fix"]].astype(int)
    sorted_df.to_csv(writePath,index=False)
#     print(i,bug_report,sorted_df.shape)
    #过滤掉topk之外的文件 取300
#     break

# 生成test数据完成

In [96]:
bug_report_files_dataframe.shape,result.shape
bug_report_files_dataframe.head()

,bid,fid,used_in_fix
0,0026caf,000f528d2d9c0bfefd8a7ea672093a0b21cabbf9,0.0
1,0026caf,004566737fca05244d8f3a9fd7b403ec6bddc453,0.0
2,0026caf,00be184c4d31b50fa6a9305ae77a3c8846d2a4a7,0.0
3,0026caf,010a7b6426963ede4625b51d6a16f26ba12a327f,0.0
4,0026caf,011d765225ccde0aa890339b7a542502fe54d342,0.0


In [113]:
len(set(result['fid']))

996

In [122]:
bug_report_files_dataframe_filter=pd.merge(sorted_df,bug_report_files_dataframe,how='left',on=['fid','bid'])

In [123]:
bug_report_files_dataframe_filter.shape

(302, 6)

In [116]:
bug_report_files_dataframe_filter.shape,result.shape,bug_report_files_dataframe.shape

((1019, 6), (1003, 5), (1374, 3))

In [ ]:
sorted_df = result.sort_values(ascending=False, by=['result'])
sorted_df=sorted_df.head(300)
# result=result.reset_index()
sorted_df.head()

In [4]:
(0.46-0.44)/0.44

0.0454545454545455

In [38]:
reportIds=all_results_df_before.index.get_level_values(0).unique()

In [2]:
def clean_str(string, TREC=False):
    string = re.sub(r"[^A-Za-z0-9(),!?;\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip() if TREC else string.strip().lower()

#
# def load_bin_vec(fname, vocab):
#     word_vecs = {}
#     model = gensim.models.KeyedVectors.load_word2vec_format(fname, binary=True)#模型的导入
#     for word in vocab:
#         if word in model.vocab:
#             word_vecs[word] = model[word]
#     return word_vecs
#
#
# def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
#     """
#     For words that occur in at least min_df documents, create a separate word vector.
#     0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
#     """
#     # for word in vocab:
#     #     if word not in word_vecs and vocab[word] >= min_df:
#     #         word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
#     i = 0  # 统计多少不在预训练词表里
#     for word in vocab:
#         if word not in word_vecs and vocab[word] >= min_df:
#             print('不在词表里词示例 ', word)
#             i += 1
#             word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
#     print('一共 ', i, ' 个词不在词表里')

def load_bin_vec(google,hdj, vocab,hdjWord2Vec):
    word_vecs = {}
    if hdjWord2Vec:
        w2v = Word2Vec.load(hdj)#模型的导入
    else:
        w2v=gensim.models.KeyedVectors.load_word2vec_format(hdj, binary=True)
    # w2v = gensim.models.KeyedVectors.load_word2vec_format(hdj, binary=True)#模型的导入
    vocab_emd=w2v.wv.vocab
    print('max_token : ', (w2v.wv.syn0.shape[0]))
    # hdj_vocab = w2v.wv.vocab
    # max_token = w2v.wv.syn0.shape[0]  # 字典的单词总量
    # print('max_token :',max_token)
    # goo_model = gensim.models.KeyedVectors.load_word2vec_format(google, binary=True)#模型的导入
    for word in vocab:
        if word in vocab_emd:
            word_vecs[word] = w2v[word]
        # else:
            # print(word,' not in 词向量')
    print('load_bin_vec :',len(word_vecs))
    return word_vecs


def add_unknown_words(word_vecs, vocab, min_df=2, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    # for word in vocab:
    #     if word not in word_vecs and vocab[word] >= min_df:
    #         word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
    i = 0  # 统计多少不在预训练词表里
    print(type(word_vecs),type(vocab))
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            # print('不在词表里词示例 ', word)
            i += 1
            word_vecs[word] = np.random.uniform(-0.25, 0.25, k)
    print('一共 ', i, ' 个词不在词表里')
def get_W(word_vecs, k=300):
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size + 1, k), dtype='float32')
    W[0] = np.zeros(k, dtype='float32')
    i = 1
    for word,emb in word_vecs.items():
        # print(type(word),word,emb)
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    print('get W ',len(W),len(word_idx_map))
    return W, word_idx_map

def getIdxfrom_sent(sent, word_idx_map, code_maxk):
    x = []
    #    pad = filter_h - 1
    #    for i in xrange(pad):
    #        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    if len(x) <= code_maxk:
        while len(x) < code_maxk:
            x.append(0)
    if len(x) >= code_maxk:
        while len(x) > code_maxk:
            x.pop()
    return x


def getIdxfrom_sent_n(sent, max_l, word_idx_map, filter_h=5):
    #TODO 为什么先pad 0 后再pad 0 ?
    x = []
    pad = filter_h - 1
    for i in range(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l + 2 * pad:
        x.append(0)
    return x


def clean_str_sst(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


# def get_W(word_vecs, k=300):
#     vocab_size = len(word_vecs)
#     word_idx_map = dict()
#     W = np.zeros(shape=(vocab_size + 1, k), dtype='float32')
#     W[0] = np.zeros(k, dtype='float32')
#     i = 1
#     for word in word_vecs:
#         W[i] = word_vecs[word]
#         word_idx_map[word] = i
#         i += 1
#     return W, word_idx_map


def alignData(data, code_maxl,code_maxk):
    try:
        mm = data.shape[0]
        nn = data.shape[1]
    except:
        tt = code_maxl 
        nn=code_maxk
        return np.zeros(nn * tt, dtype="int").reshape(tt, nn)
    if mm < code_maxl:
        tt = code_maxl - mm
        aa = np.zeros(nn * tt, dtype="int").reshape(tt, nn)
        new_data = np.vstack((data, aa))
    else:
        new_data = data[:code_maxl, :]
    return new_data

def random_permutation(train_report, train_code, labels):
    labels = np.asarray(labels)
    labels = labels.reshape(len(labels))
    datasets_size = len(labels)
    labels_rand = np.zeros(datasets_size, dtype="int")
    zz = np.random.permutation(np.arange(datasets_size))
    train_report_rand, train_code_rand = [], []
    for i in range(datasets_size):
        labels_rand[i] = labels[zz[i]]
        train_report_rand.append(train_report[zz[i]])
        train_code_rand.append(train_code[zz[i]])
    return train_report_rand, train_code_rand, labels_rand

def getReport(bug_id,report_data, maxlen,tag):
    # print(type(bug_id),type(report_data['bug_id'][0]))
    summary=None
    description=None
    # print('hhh: ',bug_id,tag)
    try:
        description=eval(report_data[report_data['bug_id'] == (bug_id)]['pos_tagged_descriptions'].values[0])['stemmed']#['unstemmed']
        # print(description)
        summary=eval(report_data[report_data['bug_id'] == (bug_id)]['pos_tagged_summarys'].values[0])['stemmed']#['unstemmed']
    except Exception as e:
        print('except : ',bug_id)
    # print(summary)
    report=None
    if description!=None and summary!=None:
        summary.extend(description)
        if len(summary) > maxlen:
            cut_words = []
            for i in range(maxlen):
                cut_words.append(summary[i])
            report = " ".join(cut_words)
        else:
            report = " ".join(summary)
    return report

# 生成report开始

In [3]:
class ReportPreprocessing:
    """Class to preprocess bug reports"""

    __slots__ = ['bug_reports']

    def __init__(self, bug_reports):
        self.bug_reports = bug_reports


    def pos_tagging(self):
        """Extracing specific pos tags from bug reports' summary and description"""

        for report in self.bug_reports.values():
            # Tokenizing using word_tokeize for more accurate pos-tagging
            summ_tok = nltk.word_tokenize(report.summary)
            try:
                desc_tok = nltk.word_tokenize(report.description)
            except Exception as e:
                print('出问题了 ',report.description)
            sum_pos = nltk.pos_tag(summ_tok)
            desc_pos = nltk.pos_tag(desc_tok)

            report.pos_tagged_summary = [token for token, pos in sum_pos
                                         if 'NN' in pos or 'VB' in pos]
            report.pos_tagged_description = [token for token, pos in desc_pos
                                             if 'NN' in pos or 'VB' in pos]

    def tokenize(self):
        """Tokenizing bug reports into tokens"""

        for report in self.bug_reports.values():
            report.summary = nltk.wordpunct_tokenize(report.summary)
            report.description = nltk.wordpunct_tokenize(report.description)

    def _split_camelcase(self, tokens):

        # Copy tokens
        returning_tokens = tokens[:]

        for token in tokens:
            split_tokens = re.split(fr'[{string.punctuation}]+', token)

            # If token is split into some other tokens
            if len(split_tokens) > 1:
                returning_tokens.remove(token)
                # Camel case detection for new tokens
                for st in split_tokens:
                    camel_split = inflection.underscore(st).split('_')
                    if len(camel_split) > 1:
                        returning_tokens.append(st)
                        returning_tokens += camel_split
                    else:
                        returning_tokens.append(st)
            else:
                camel_split = inflection.underscore(token).split('_')
                if len(camel_split) > 1:
                    returning_tokens += camel_split

        return returning_tokens

    def split_camelcase(self):
        """Split CamelCase identifiers"""

        for report in self.bug_reports.values():
            report.summary = self._split_camelcase(report.summary)
            report.description = self._split_camelcase(report.description)
            report.pos_tagged_summary = self._split_camelcase(report.pos_tagged_summary)
            report.pos_tagged_description = self._split_camelcase(report.pos_tagged_description)

    def normalize(self):
        """Removing punctuation, numbers and also lowercase conversion"""

        # Building a translate table for punctuation and number removal
        punctnum_table = str.maketrans({c: None for c in string.punctuation + string.digits})

        for report in self.bug_reports.values():
            summary_punctnum_rem = [token.translate(punctnum_table)
                                    for token in report.summary]
            desc_punctnum_rem = [token.translate(punctnum_table)
                                 for token in report.description]
            pos_sum_punctnum_rem = [token.translate(punctnum_table)
                                    for token in report.pos_tagged_summary]
            pos_desc_punctnum_rem = [token.translate(punctnum_table)
                                     for token in report.pos_tagged_description]

            report.summary = [token.lower() for token
                              in summary_punctnum_rem if token]
            report.description = [token.lower() for token
                                  in desc_punctnum_rem if token]
            report.pos_tagged_summary = [token.lower() for token
                                         in pos_sum_punctnum_rem if token]
            report.pos_tagged_description = [token.lower() for token
                                             in pos_desc_punctnum_rem if token]

    def remove_stopwords(self):
        """Removing stop words from tokens"""

        for report in self.bug_reports.values():
            report.summary = [token for token in report.summary
                              if token not in stop_words]
            report.description = [token for token in report.description
                                  if token not in stop_words]
            report.pos_tagged_summary = [token for token in report.pos_tagged_summary
                                         if token not in stop_words]
            report.pos_tagged_description = [token for token in report.pos_tagged_description
                                             if token not in stop_words]

    def remove_java_keywords(self):
        """Removing Java language keywords from tokens"""

        for report in self.bug_reports.values():
            report.summary = [token for token in report.summary
                              if token not in java_keywords]
            report.description = [token for token in report.description
                                  if token not in java_keywords]
            report.pos_tagged_summary = [token for token in report.pos_tagged_summary
                                         if token not in java_keywords]
            report.pos_tagged_description = [token for token in report.pos_tagged_description
                                             if token not in java_keywords]

    def stem(self):
        """Stemming tokens"""

        # Stemmer instance
        stemmer = PorterStemmer()

        for report in self.bug_reports.values():
            report.summary = dict(zip(['stemmed', 'unstemmed'],
                                      [[stemmer.stem(token) for token in report.summary],
                                       report.summary]))

            report.description = dict(zip(['stemmed', 'unstemmed'],
                                          [[stemmer.stem(token) for token in report.description],
                                           report.description]))

            report.pos_tagged_summary = dict(zip(['stemmed', 'unstemmed'],
                                                 [[stemmer.stem(token) for token
                                                   in report.pos_tagged_summary],
                                                  report.pos_tagged_summary]))

            report.pos_tagged_description = dict(zip(['stemmed', 'unstemmed'],
                                                     [[stemmer.stem(token) for token
                                                       in report.pos_tagged_description],
                                                      report.pos_tagged_description]))

    def preprocess(self):
        """Run the preprocessing"""

        self.pos_tagging()
        self.tokenize()
        self.split_camelcase()
        self.normalize()
        self.remove_stopwords()
        self.remove_java_keywords()
        self.stem()
class BugReport:
    """Class representing each bug report""" 
    
    __slots__ = ['summary', 'description',
                 'pos_tagged_summary', 'pos_tagged_description']
    
    def __init__(self, summary, description):
        self.summary = summary
        self.description = description
        self.pos_tagged_summary = None
        self.pos_tagged_description = None
def load_bug_reports(bug_report_file_path):
    """load bug report file (the one generated from xml)"""
    with open(bug_report_file_path) as bug_report_file:
        bug_reports = json.load(bug_report_file)
        return bug_reports


In [77]:
# bug_report_file_path='/data/hdj/tracking_buggy_files/'+swt+'.json'
# bug_report_file_path='/data/hdj/tracking_buggy_files/eclipse/eclipse_platform_ui.json'
project='swt'
bug_report_file_path='/data/hdj/tracking_buggy_files/'+project+'/'+project+'.json'
bug_reports = load_bug_reports(bug_report_file_path)
bug_reports_new = dict()

for i,(key,val)  in enumerate(bug_reports.items()):
    bug_reports_new[key] = BugReport(
        ' '.join(val['bug_report']['summary'].split()[2:]),
        '' if  val['bug_report']['description']==None else val['bug_report']['description']
    )
report_prep = ReportPreprocessing(bug_reports_new)
report_prep.preprocess()
summarys=[]
descriptions=[]
pos_tagged_summarys=[]
pos_tagged_descriptions=[]
bug_ids=[]
for key,report in zip(report_prep.bug_reports.keys(),report_prep.bug_reports.values()):
    summarys.append(report.summary)
    descriptions.append(report.description)
    pos_tagged_summarys.append(report.pos_tagged_summary)
    pos_tagged_descriptions.append(report.pos_tagged_description)
    bug_ids.append(key)
report=pd.DataFrame()
report['bug_id']=bug_ids
report['pos_tagged_descriptions']=pos_tagged_descriptions
report['pos_tagged_summarys']=pos_tagged_summarys
report['descriptions']=descriptions
report['summarys']=summarys
# report.to_csv('/data/hdj/cross_project_trans/report/bert/'+project+'_report_clean.csv',index=False)


In [80]:
#验证一下正确性
report_data=pd.read_csv('/data/hdj/cross_project_trans/report/bert/'+project+'_report_clean.csv')
report_data.head()

,bug_id,pos_tagged_descriptions,pos_tagged_summarys,descriptions,summarys
0,bae8b23,"{'stemmed': ['red', 'hat', 'eclips', 'open', '...","{'stemmed': ['dcr', 'need', 'display', 'getsys...","{'stemmed': ['red', 'hat', 'eclips', 'open', '...","{'stemmed': ['dcr', 'need', 'display', 'getsys..."
1,32055ba,"{'stemmed': ['computes', 'method', 'appear', '...","{'stemmed': ['label', 'requir', 'draw'], 'unst...","{'stemmed': ['computes', 'method', 'appear', '...","{'stemmed': ['label', 'requir', 'border', 'dra..."
2,9d45068,"{'stemmed': ['run', 'eclips', 'grab', 'sash', ...","{'stemmed': ['sash', 'cursor', 'chang', 'drag'...","{'stemmed': ['run', 'eclips', 'grab', 'sash', ...","{'stemmed': ['sash', 'cursor', 'chang', 'drag'..."
3,1ee8264,"{'stemmed': ['build', 'identifi', 'notic', 'wo...","{'stemmed': ['window', 'render', 'restor', 'mi...","{'stemmed': ['build', 'identifi', 'notic', 'wo...","{'stemmed': ['workbench', 'window', 'render', ..."
4,f35602d,"{'stemmed': ['isnt', 'copi', 'past', 'text', '...","{'stemmed': ['browser', 'copi', 'past', 'text'...","{'stemmed': ['isnt', 'possibl', 'copi', 'past'...","{'stemmed': ['browser', 'unabl', 'copi', 'past..."


# 生成report信息结束

# 生成code字典开始

In [4]:
def clean_string_code(string):
    m = re.compile(r'/\*.*?\*/', re.S)
    outstring = re.sub(m, '', string)
#     m = re.compile(r'package*', re.S)
#     outstring = re.sub(m, '', outstring)
#     m = re.compile(r'import*', re.S)
#     outstring = re.sub(m, '', outstring)
    m = re.compile(r'//.*')
    outtmp = re.sub(m, '', outstring)
    
    return outtmp
# fid='7dbde289a9c63fbe8464de0e913442e6889b57bf'
# ast_raw_file = pickle.loads(ast_cache_collection_db['7dbde289a9c63fbe8464de0e913442e6889b57bf'])
# rawContent=clean_string_code(ast_raw_file['rawSourceContent'])
# codeWritePath=os.path.join(codePath,fid+'.java')
# with open(codeWritePath,'w',encoding='utf-8') as f_out:
#     f_out.write(rawContent)

In [128]:
# rawContent

In [123]:
fidList.pop()

'7dbde289a9c63fbe8464de0e913442e6889b57bf'

In [47]:
ast_cache_collection_db=UnQLite("/data/hdj/tracking_buggy_files/"+project+'/'+project+"_ast_cache_collection_db",flags=0x00000100 | 0x00000001)
outPath='/data/hdj/cross_project_trans/report/bert/'
train_pos=pd.read_csv('/data/hdj/cross_project_trans/report/bert/'+project+'_80_training_pos.csv')
train_neg=pd.read_csv('/data/hdj/cross_project_trans/report/bert/'+project+'_80_training_neg.csv')
fidList=set()
fidList=fidList.union(train_pos['fid'].values)
fidList=fidList.union(train_neg['fid'].values)
print(len(fidList))
testNum=len(os.listdir('/data/hdj/cross_project_trans/report/bert/swt_test/'))
for i in range(500):
    filePath=os.path.join(outPath,project+'_test','test'+str(i)+'.csv')
#     print(filePath)
    data=pd.read_csv(filePath)
    fidList=fidList.union(data['fid'].values)
print(len(fidList))    
# codeDict=dict()
codePath='/data/hdj/cross_project_trans/report/bert/'+project+'_code'
for fid in fidList:
#     print(fid)
    ast_raw_file = pickle.loads(ast_cache_collection_db[fid])
#     codeDict[fid]=ast_raw_file['rawSourceContent']
    rawContent=clean_string_code(ast_raw_file['rawSourceContent'])
    codeWritePath=os.path.join(codePath,fid)
    with open(codeWritePath,'w',encoding='utf-8') as f_out:
        f_out.write(rawContent)
#     break

# for fid,rawContent in codeDict.items():
    

11285
14483


In [5]:
def clean_str(string, TREC=False):
    string = re.sub(r"[^A-Za-z0-9(),!?;\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip() if TREC else string.strip().lower()



In [120]:
clean_str('LRESULT wmScrollChild (int /*long*/ wParam, int /*long*/ lParam) {')

'lresult wmscrollchild \\( int long wparam , int long lparam \\)'

# 生成code字典结束

# 对code进行清洗开始

In [6]:
class SrcPreprocessing_fulltext:
    def __init__(self,code_dir):
        self.src_files=self.get_repo_source_files(code_dir)

    def tokenize(self):
        """Tokenizing source codes into tokens"""
        for key,vals in self.src_files.items():
            tmp=[]
            for val in vals:
                tmp.append(nltk.wordpunct_tokenize(val))
            self.src_files[key]= tmp

    def _split_camelcase(self,tokens):

        # Copy tokens
        returning_tokens = tokens[:]

        for token in tokens:
            split_tokens = re.split(fr'[{string.punctuation}]+', token)

            # If token is split into some other tokens
            if len(split_tokens) > 1:
                returning_tokens.remove(token)
                # Camel case detection for new tokens
                for st in split_tokens:
                    camel_split = inflection.underscore(st).split('_')
                    if len(camel_split) > 1:
                        returning_tokens.append(st)
                        returning_tokens += camel_split
                    else:
                        returning_tokens.append(st)
            else:
                camel_split = inflection.underscore(token).split('_')
                if len(camel_split) > 1:
                    returning_tokens += camel_split

        return returning_tokens

    def split_camelcase(self):
        """Split CamelCase identifiers"""

        for key,val_list in self.src_files.items():
            tmp = []
            # print('vals',val_list)
            for vals in val_list:
                # print('vals',vals)
                tmp.append(self._split_camelcase(vals))
            self.src_files[key] = tmp
    def normalize(self):
        """Removing punctuation, numbers and also lowercase conversion"""

        # Building a translate table for punctuation and number removal
        punctnum_table = str.maketrans({c: None for c in string.punctuation + string.digits})

        for key,vals in self.src_files.items():
            tmp = []
            for val in vals:
                # print(val)
                content_punctnum_rem = [token.translate(punctnum_table)for token in val]
                tmp.append([token.lower() for token in content_punctnum_rem if token])
            self.src_files[key] = tmp
    def remove_stopwords(self):
        """Removing stop words from tokens"""

        for key, val_list in self.src_files.items():
            tmp=[]
            for val in val_list:
                after_removed=[token for token in val if token not in stop_words]
                if len(after_removed)>0:
                    tmp.append(after_removed)
            self.src_files[key] = tmp
    def remove_java_keywords(self):
        """Removing Java language keywords from tokens"""

        for key, val_list in self.src_files.items():
            tmp=[]
            for val in val_list:
                after_removed =[token for token in val  if token not in java_keywords]
                if len(after_removed) > 0:
                    tmp.append(after_removed)
            self.src_files[key] = tmp

    def stem(self):
        """Stemming tokens"""

        # Stemmer instance
        stemmer = PorterStemmer()

        for key, val_list in self.src_files.items():
            tmp = []
            for val in val_list:
                tmp.append(dict(zip(['stemmed', 'unstemmed'],[[stemmer.stem(token) for token in val], val])))
            self.src_files[key] = tmp
    def preprocess(self):
        """Run the preprocessing"""

        # self.pos_tagging()
        print('tokenize')
        self.tokenize()
        print('split_camelcase')
        self.split_camelcase()
        print('normalize')
        self.normalize()
#         print('remove_stopwords')
#         self.remove_stopwords()
#         print('remove_java_keywords')
#         self.remove_java_keywords()
        print('stem')
        self.stem()
    def get_repo_source_files(self,code_dir):
        files = dict()
        start_dir = os.path.normpath(code_dir)  # 规范path字符串形式
        for dir_, dir_names, file_names in os.walk(start_dir):  # os.walk()方法用于通过在目录树中游走输出在目录中的文件名，向上或者向下
#             for filename in [f for f in file_names if f.endswith(".java")]:
            for i,filename in enumerate(file_names) :
#                 if i==10:
#                     break
                src_name = os.path.join(dir_, filename)  # 所有java文件名
                with open(src_name, encoding='utf-8', mode='r') as src_file:
                    lines = src_file.readlines()
                file_key = src_name.split(start_dir)[1]
                file_key = file_key[len(os.sep):]  # os.sep 不用考虑是linux 的‘/’ 或者是windows的 ‘\’

                # 过滤掉 copyright
                new_lines = []
                beforeTag = True
                for line in lines:
                    line = line.lstrip().replace('\n', '')
                    if 'package' in line or 'import' in line or 'class' in line:
                        beforeTag = False
                    if beforeTag == False:
                        new_lines.append(line)

                # 分离code和description
                source = []
                # if file_key == 'org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                #     print('newline: ',new_lines)
                for line in new_lines:
                    # if file_key == 'org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                    #     print(line)
                    if line == "": continue
                    if line.startswith("@"): continue
                    if line.find('/*') >= 0: continue
                    if line.startswith('*'): continue
                    if line.find('*/') >= 0: continue
                    # if line.find('import')>=0:continue
                    if line.startswith('//'):
                        continue
                    else:
                        if line.find('//') >= 0:
                            index = line.index('//')
                            line = line[:index]
                        # source += (line + '\n')
                        source.append(line + '\n')

                # print(source,'\n******************************\n')
                # print(file_key)
                # if file_key=='org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                #     print('here ',file_key,source)
                files[file_key] = source
        return  files

In [94]:
code_prep.stem()

In [7]:
import glob
srcs={'/data/hdj/cross_project_trans/report/bert/swt_code':'/data/hdj/cross_project_trans/report/bert/swt_code_clean'}

for src ,clean_path in srcs.items():
    src_addresses = glob.glob(src + '/*', recursive=True)
    code_prep = SrcPreprocessing_fulltext(src)
    code_prep.preprocess()
    print('总共java文件数 ，处理后java文件数',len(src_addresses),len(code_prep.src_files))
    j=0
    for path,val_list in code_prep.src_files.items():
        
#         path=pathNorm(path)
#         path=os.path.join(clean_path[i],path)
        path=os.path.join(clean_path,path)
#         print(path)
#         mkdir(path)
#         if len(val_list)==0:
#             print(path,' is null')
#             j+=1
#             continue
        with open(path, 'w', encoding='utf-8') as f_out:
            for val in val_list:
                f_out.write(' '.join(val['stemmed']) + '\n')
#     print(reg,'共有',j,'个文件被过滤掉')
    i+=1

tokenize
split_camelcase


KeyboardInterrupt: 

In [95]:
for path,val_list in code_prep.src_files.items():
    path=os.path.join(clean_path,path)

    with open(path, 'w', encoding='utf-8') as f_out:
        for val in val_list:
            f_out.write(' '.join(val['stemmed']) + '\n')

# 对code进行清洗完成

In [7]:
def load_data( code_maxl, code_maxk, report_maxl, w2v_file, report_path,project_name,more_project_name,less_project_name,clean_path,hdjWord2vec):
    '''
        code_maxl:代码的最大行数 100
        code_maxk:代码每行最大单词数 40 
        report_maxl:report最大的单词数 308
        w2c_file: 词向量所在位置/data/hdj/cross_project_trans/embedding/word_w2v_100
        report_path:/data/hdj/cross_project_trans/report/bert 文件保存所在位置
        project_name:将要处理的项目aspectj_
        more_project_name:aspectj_80_
        less_project_name:aspectj_20_
        clean_path:清洗过后的项目路径/data/hdj/cross_project_trans/report/bert/swt_code_clean
        
    '''
    less_report_ids=[]
    #revs:保存report的dict信息
    '''
    #revs_code:保存一个code文件的所有行 [
                            [{"y": 1,#这是一行的数据
                             "text": orig_rev,
                             "num_words": len(orig_rev.split()),
                             "test": -1,
                             "target": -1,
                             "path": source_code_name,
                             'bug_id': bug_report_id},{},{}这是很多行
                         ]
                        ]
    '''
    
    report_ids,revs, revs_code = [], [],[]
    report, code, train_path = [], [], []
    revs_pre = []
    vocab = defaultdict(float)
    #拼接成的路径 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv
    #/data/hdj/cross_project_trans/report/bert/swt_report_clean.csv
    report_data=pd.read_csv(report_path+project_name+'report_clean.csv')
    print('读取80% training_pos的数据')#读取80% training_pos的数据/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv
    spamreader = csv.reader(open(report_path+more_project_name+"training_pos.csv", newline=''))
    i=0
    for row in spamreader:
        if i==0:
            i+=1
            continue
        #根据bugid获得对应report
        bug_report_id = row[0].strip()
        report_ids.append(bug_report_id)#TODO 记录report_id
        bug_description = getReport(bug_report_id,report_data, report_maxl,'ll')
        # report信息获得完成
        # 统计词表信息
        orig_rev = bug_description.strip().lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        #统计完成
        #将report相关信息保存
        datum = {"y": 1,#是正例
                 "text": orig_rev,#report内容
                 "num_words": len(orig_rev.split()),#report长度
                 "test": -1,#不是测试数据
                 "target": -1,
                 'bug_id': bug_report_id}
        revs.append(datum)#TODO 记录report
        #保存完成

        source_code_name = row[1].strip()
        #clean_path+source_code_name='/home/hdj/bug_localization/data/ZXing/clean/' + 'com/google/zxing/client/rim/util/Log.java'
        #/data/hdj/cross_project_trans/report/bert/jdt_code
        with open(os.path.join(clean_path,source_code_name), "r") as f:
            revs_single = []#保存一个java文件所有行[[import pandas as pd],[],[]]
            for k,line in enumerate(f):
                orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
                if len(orig_rev) <= 1: continue#这里过滤了数量小于1的行
                words = set(orig_rev.split())
                for word in words:
                    vocab[word] += 1
                datum = {"y": 1,
                         "text": orig_rev,
                         "num_words": len(orig_rev.split()),
                         "test": -1,
                         "target": -1,
                         "path": source_code_name,
                         'bug_id': bug_report_id}
                revs_single.append(datum)
                if len(revs_single)>=code_maxl:
                    break
            revs_code.append(revs_single)#保存所有的java文件内容 [[revs_single],[]] #TODO 记录该report对应的code
    # 读取80% training_pos的数据完成
    print('读取80% training_pos的数据完成')


    print('读取80% training negative 的数据')
    #读取80% training negative 的数据
    spamreader = csv.reader(open(report_path+more_project_name+"training_neg.csv", newline=''))
    i=0
    for time,row in enumerate(spamreader):
        if i==0:
            i+=1
            continue
       # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
        
        bug_report_id = row[0].strip()
        if time%10000==0:
            print('train neg: ',bug_report_id)
        report_ids.append(bug_report_id)#TODO 记录report_id

        #统计词频
        bug_description = getReport(bug_report_id, report_data, report_maxl,'aa')
        # rev = []
        # rev.append(bug_description.strip())
        # orig_rev = " ".join(rev).lower()
        orig_rev = bug_description.strip().lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        #统计完成


        datum = {"y": 0,
                 "text": orig_rev,
                 "num_words": len(orig_rev.split()),
                 "test": -1,
                 "target": -1,
                 "bug_id":bug_report_id}
        revs.append(datum)#TODO 记录report

        source_code_name = row[1].strip()
        with open(os.path.join(clean_path,source_code_name), "r") as f:
            revs_single = []
            for k,line in enumerate(f):
                orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
                if len(orig_rev) <= 1: continue
                words = set(orig_rev.split())
                for word in words:
                    vocab[word] += 1
                datum = {"y": 0,
                         "text": orig_rev,
                         "num_words": len(orig_rev.split()),
                         "test": -1,
                         "target": -1,
                        "path":source_code_name,
                         "bug_id": bug_report_id}
                revs_single.append(datum)
                if len(revs_single)>=code_maxl:
                    break
            if len(revs_single)==0:#TODO 注意 过滤后的java文件不应该为空，否则后续处理有问题
                print('this is the bug')
            revs_code.append(revs_single)#TODO 记录该report对应的code
    # 读取80% training negative 的数据完成
    print('读取80% training negative 的数据完成')

    # 读取test数据  test_num ：number of test bug reports #TODO 数量不固定，提前传个参数
    #test_path=report_path+project_name+'test/test'+str(index)+'.csv'
    #  例如    '/home/hdj/cross_project_trans/report/'+'aspectj_'+'test/test'+'0'+'.csv'
    test_path=report_path+project_name+'test/'
#     test_num=len(os.listdir(test_path))
    test_num=100
    print('读取test数据',test_num)
    for test_index in range(test_num):
        print('test num :',test_index)
        spamreader = csv.reader(open(report_path+project_name+'test/test' + str(test_index) + ".csv", newline=''))
        i=0
        for row in spamreader:
            if i == 0:
                i += 1
                continue
            label = row[2]
            # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
            bug_report_id = row[0].strip()
            # report_ids.append(bug_report_id)#TODO 记录test 的report_id

            bug_description = getReport(bug_report_id, report_data, report_maxl,'dd')
            orig_rev = bug_description.strip().lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            if label.strip() == "1":
                datum = {"y": 1,
                         "text": orig_rev,
                         "num_words": len(orig_rev.split()),
                         "test": test_index,#用来记录test索引 每个测试report都单独处理
                         "target": 1,
                         "bug_id": bug_report_id}
            else:
                datum = {"y": 0,
                         "text": orig_rev,
                         "num_words": len(orig_rev.split()),
                         "test": test_index,
                         "target": 1,
                         "bug_id": bug_report_id}
            #revs 存放report信息
            revs.append(datum)#TODO 记录test report

          #  source_file_name = "JDT_CNN/sources/sources/JDT_4_5/" + row[1].strip()
            source_code_name = row[1].strip()
            # print(os.path.join(pre, source_file_name))
            with open(os.path.join(clean_path,source_code_name), "r") as f:
                revs_single = []
                for k,line in enumerate(f):
                    orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
                    if len(orig_rev) <= 1:
                        orig_rev = "test"
                    words = set(orig_rev.split())
                    for word in words:
                        vocab[word] += 1
                    if label.strip() == "1":
                        datum = {"y": 1,
                                 "text": orig_rev,
                                 "num_words": len(orig_rev.split()),
                                 "test": test_index,
                                 "target": 1,
                                 "path":source_code_name,
                                 "bug_id": bug_report_id}
                    else:
                        datum = {"y": 0,
                                 "text": orig_rev,
                                 "num_words": len(orig_rev.split()),
                                 "test": test_index,
                                 "target": 1,
                                 "path": source_code_name,
                                 "bug_id": bug_report_id}#TODO 根据这个区匹配对应的AST
                    revs_single.append(datum)
                    if len(revs_single)>=code_maxl:
                        break
                if len(revs_single) == 0:
                    revs_single = revs_pre
                if len(revs_single)==0:
                    print(os.path.join(clean_path, source_code_name))
                    print('here null :',revs_single,' llllll')
                revs_code.append(revs_single) #TODO 记录该report对应的code
                revs_pre = revs_single
    print('读取test数据 ', test_num,' 完成')
    w2v = load_bin_vec('',w2v_file, vocab,hdjWord2vec)
    if hdjWord2vec:
        add_unknown_words(w2v, vocab,k=100)
        W, word_idx_map = get_W(w2v,k=100)
    else:
        add_unknown_words(w2v, vocab)
        W, word_idx_map = get_W(w2v)
    print('这里显示向量真实大小')
    print(len(W),len(W[0]))
    #对train80% 数据进行整理
    print('对train80% 数据进行整理')
    train_id,train_report, train_code, train_labels = [], [], [],[]
    #1. 对report信息进行操作
    for rev in revs:
        
        if rev["test"] == -1:
            sent = getIdxfrom_sent_n(rev["text"], report_maxl, word_idx_map, filter_h=5)
            train_id.append(rev["bug_id"])
            train_report.append(sent)
            report.append(rev['text'])
    #2. 对code信息进行操作
    for revs_single in revs_code:
        data_single = []
        label_single = []
        if len(revs_single)==0:
            print('null ',revs_single,' kl')
        for rev in revs_single:
                sent = getIdxfrom_sent(rev["text"], word_idx_map, code_maxk=code_maxk)
                data_single.append(sent)
        if rev["test"] == -1:
            
            data_single = np.array(data_single, dtype="int")
            if len(data_single)==0:
                print(type(data_single),data_single,revs_single)
#             try:
            data_single = alignData(data_single, code_maxl,code_maxk)
            train_code.append(data_single)
            label_single.append(rev["y"])
            train_labels.append(label_single)
            code.append(revs_single)
            train_path.append(rev['path'])
#             except:
#                 continue

    train_labels = np.array(train_labels, dtype="int")
    #train80% 数据整理完成
    print('对train80% 数据进行整理完成')

    #对测试集进行整理
    print('对测试集进行整理')
    test_report, test_code, test_path, test_bug_id, test_raw_code, test_row_report = [], [], [], [], [], []
    test_labels = []
    #TODO 这里test_num也需要根据项目来指定
    for i in range(test_num):
        test_report.append([])
        test_code.append([])
        test_labels.append([])
        test_path.append([])
        test_bug_id.append([])
        test_raw_code.append([])
        test_row_report.append([])
    #1. report进行整理
    for rev in revs:
        # sent = []
        index = rev["test"]
        if index >= 0:
            sent = getIdxfrom_sent_n(rev["text"], report_maxl, word_idx_map, filter_h=5)
            test_bug_id[index].append(rev['bug_id'])
            test_report[index].append(sent)
            test_row_report[index].append(rev['text'])
    #report 整理完成

    #2. 对test code进行整理
    for revs_single in revs_code:
        data_single = []
        label_single = []
        for rev in revs_single:
            sent = getIdxfrom_sent(rev["text"], word_idx_map, code_maxk=code_maxk)
            data_single.append(sent)

        index = rev["test"]
        if index >= 0:
            label_single.append(rev["y"])
            data_single = np.array(data_single, dtype="int")
            data_single = alignData(data_single, code_maxl,code_maxk)
            test_code[index].append(data_single)
            test_labels[index].append(label_single)
            test_path[index].append(rev['path'])
            test_raw_code[index].append(revs_single)
    #test code整理完成

    #single_path='singleProject/'
    single_path=''
    #3. 将test数据保存
    print('*' * 10)
    print('test bug id len', len(test_bug_id), ' test report : ', len(test_report), ' test code ', len(test_code),
          ' test label ', len(test_labels), ' test path ', len(test_path), 'test_row_report ', len(test_row_report)
          , ' test_raw_code ', len(test_raw_code))
#     ast_path='/data/hdj/cross_project_trans/out/'+project_name+'tree_blocks.pkl'
#     ast_file = pd.read_pickle(ast_path)
#     ast_file = ast_file[['path', 'ast_code']]#TODO 将ast生成时字段改为ast_code
    for i in range(test_num):
        test_labels[i] = np.array(test_labels[i], dtype="int")
        test_data = pd.DataFrame()
        #300 300 0 0 0 300 0
        print(len(test_bug_id[i]),len(test_report[i]),len(test_code[i]),len(test_labels[i]),len(test_path[i]),len(test_row_report[i]),len(test_raw_code[i]))
        test_data['bug_id'] = test_bug_id[i]
        test_data['cnn_report_idx'] = test_report[i]
        test_data['code_idx'] = test_code[i]
        test_data['label'] = test_labels[i]
        test_data['path'] = test_path[i]
        test_data['raw_report'] = test_row_report[i]
        test_data['raw_code'] = test_raw_code[i]
        # 合并ast数据
#         test_data = pd.merge(test_data, ast_file, on='path', how='left')
        test_data.to_pickle(report_path+single_path+project_name+'test_middle/test' + str(i) + '.pkl')
    #test数据保存完成
    #测试集整理完成
    #80% train数据保存
    print('vaocab ', len(vocab))
    print('word_idx_map : ', len(word_idx_map))
    # print(report_ids[0])
    # print(train_report[0])
    # print(train_code[0])
    # print(train_labels[0])
    # print(report[0])
    # print(code[0])
    # print(train_path[0])
    train_data = pd.DataFrame()
    print('数据长度 : ', len(report_ids), len(train_report), len(train_code), len(train_labels), len(report), len(code),
          len(train_path))
    train_data['bug_id'] = report_ids
    train_data['cnn_report_idx'] = train_report
    train_data['code_idx'] = train_code
    train_data['label'] = train_labels
    train_data['raw_report'] = report
    train_data['raw_code'] = code
    train_data['path'] = train_path
#     train_data = pd.merge(train_data, ast_file, on='path', how='left')
    train_data.to_pickle(report_path+single_path+project_name+'more_train_data.pkl')
    #80% 数据保存完成
    print("Finish loading")

    pickle.dump([W], open(report_path+single_path + project_name + 'test_middle/'+"w_parameters.in", "wb"))
    return train_report, train_code, train_labels, W

In [18]:
files=os.listdir('/data/hdj/cross_project_trans/report/bert/swt_code')

In [20]:
files[:10]

['17009903228987a63b0f317b752f7d5f136fcf85',
 'edf6dc88b977718d5cf768f38193840bde20c33f',
 '75d04d2f9252d2557efc8ae75e4e476a2080f663',
 '81ae7ead5941e365d02039789515e2f4c3a2d23a',
 '3357f54116a2bd82e8a5098bfd70beb4aaa9ae0a',
 '2d3223c18b6945e4b1f1b72ad4b2a5cb2767f4d9',
 'c1253bc85daff5852283c90457cc5409c38fe0bc',
 '271aa45addedbab6945a9be77d10af7b7b9d7d17',
 'af9c1d5f4856c3524dd774ffef83ac13218bf36c',
 '7f755fa34edcabcf275f067327751101a9a185f3']

In [21]:
# print(os.system('wc -l /data/hdj/cross_project_trans/report/bert/swt_code/0a0c755a207bb828ca75cd6667cc87fd2d23a29f'))
import os
path='/data/hdj/cross_project_trans/report/bert/swt_code'
totalLen=0
for file in files:
    
    command = 'wc -l '+os.path.join(path,file) #可以直接在命令行中执行的命令
    r = os.popen(command) #执行该命令
    info = r.readlines()  #读取命令行的输出到一个list
#     print(type(info),len(info))
    line = info[0].strip('\r\n')
    contents=line.split(' ')
    totalLen+=int(contents[0])
    print(contents[0])

419
325
1019
165
482
4399
1212
363
234
1481
18
1609
99
44
379
48
349
947
22
1497
1062
245
61
552
3173
6405
3016
3084
304
487
202
792
333
541
1506
325
218
2422
252
1065
524
165
27
766
2738
1223
93
88
750
61
2874
998
319
3476
5485
1270
616
2640
142
415
287
725
1244
2423
3139
114
15
225
201
1180
97
16
83
12466
1784
424
805
142
2569
503
1215
3066
829
158
1138
2670
860
818
1837
752
2438
584
490
147
142
2687
13626
2597
1003
262
134
189
211
89
4628
673
3340
26
1801
765
38
583
10
2420
2225
1240
791
176
737
696
1112
188
80
2660
1622
5699
12244
581
2467
1613
52
2748
5451
686
624
2857
48
761
330
268
257
167
117
84
454
377
175
1572
1212
1506
755
23
175
2575
31
321
2352
359
244
595
49
707
569
1538
223
5292
216
319
265
608
3583
346
48
2916
384
894
996
720
546
565
24
829
100
1247
336
141
153
68
657
889
454
398
55
2440
3021
446
21
1716
139
600
4980
3198
1417
1364
560
1107
2106
485
1184
320
612
177
3944
519
225
1472
1666
3156
430
99
796
1217
418
1216
144
414
342
121
1260
429
758
405
818
961
583
433
225

95
2470
4242
1504
350
49
127
192
230
662
379
1180
1801
656
299
1003
2591
1858
455
340
105
1659
508
1590
1852
1283
26
2246
35
159
897
2907
475
595
764
3871
557
320
719
2024
1894
620
17
293
200
661
4486
232
87
1915
1472
1057
159
7010
288
699
739
17
1574
672
1366
6988
1188
168
725
1663
973
501
291
2864
208
860
1661
734
53
229
1421
1065
511
358
123
145
111
714
1342
2110
5352
1576
260
257
30
166
59
312
952
535
557
49
1058
65
71
56
1884
173
498
520
1155
141
3138
206
326
453
94
26
1323
4975
335
2214
1605
366
135
107
63
450
306
611
157
91
133
2761
481
824
735
384
160
965
2910
2158
859
11936
582
113
268
637
2320
1401
628
811
707
797
292
224
105
1474
718
1781
61
340
1624
1229
790
2322
188
63
6
1902
883
6220
2405
212
708
2689
965
41
15
1044
3307
62
2325
256
414
1155
1863
2480
483
1370
182
869
186
286
60
40
2532
77
16
1795
1363
1269
1508
1236
1551
56
586
1585
2128
1193
192
959
761
162
111
1545
4313
70
365
1830
303
807
1200
4799
572
1319
3263
389
606
839
2346
311
1946
569
1827
1551
244
38
1140
176


416
1513
397
93
687
253
85
4756
817
1714
481
769
37
1323
1128
573
177
2680
45
601
441
1242
74
42
2451
132
1749
210
272
874
261
116
307
225
685
1323
2112
213
1912
875
1278
1009
4951
17
49
12
335
1839
155
176
1712
746
110
1666
38
3048
265
404
102
591
382
66
4855
1529
2760
2660
2403
365
1756
18
3788
66
301
3462
1156
631
128
173
52
85
795
249
320
394
942
515
3213
56
58
458
292
2348
1679
465
35
430
958
2089
6532
453
1655
410
219
232
1710
193
1847
2498
128
2778
354
5348
1376
2625
1184
240
2472
406
479
78
995
269
576
123
792
2257
650
1659
491
1200
245
75
766
5306
345
389
140
111
2581
13025
597
1114
115
393
661
810
2923
8253
2625
436
556
874
985
1472
85
738
711
700
65
826
1596
147
185
1134
122
519
144
1585
363
710
12
1452
2406
1018
295
1658
42
26
107
247
3350
121
1339
653
247
3722
2736
1105
182
53
5279
422
244
662
1010
2634
326
12
2531
165
3091
150
195
949
865
222
8579
556
1046
198
260
3335
371
395
131
66
983
32
787
1225
2902
3476
544
472
2686
12941
4808
3321
692
175
898
308
161
8
10433
488
39

80
309
1570
5261
2632
690
1680
160
1017
24
1413
4677
1338
54
200
1260
931
472
441
364
1625
308
24
506
860
275
135
580
545
539
268
87
386
220
301
463
276
249
17
129
219
1415
3790
2170
789
857
62
588
286
507
845
264
293
47
134
7089
396
905
4398
165
930
464
157
327
2607
210
100
352
1662
1400
217
209
1041
113
1720
720
750
168
43
490
378
79
51
597
343
125
26
176
564
2752
190
295
845
3560
570
3031
379
12
2115
472
4137
884
692
1419
780
60
340
681
406
410
371
1020
270
169
463
12478
876
2753
1617
416
2537
830
58
170
2346
609
5863
6590
5417
3028
407
92
3079
1407
162
701
14
205
2783
144
315
562
3821
604
56
2405
305
2916
3815
3320
1226
1311
441
1549
917
299
1153
790
658
2762
2818
192
12921
311
551
1691
2753
500
184
1565
486
946
228
1589
2200
902
305
434
1309
43
317
922
341
779
86
170
807
886
2490
369
3530
623
1294
65
1472
367
137
1033
1052
63
748
183
654
35
2714
307
155
1411
1067
2700
2569
160
140
74
758
2185
12144
1368
505
776
41
12
1655
86
6407
180
640
490
1250
1866
2869
82
1326
2525
654
514
20


423
480
864
469
578
2337
562
2184
116
800
315
367
31
802
110
162
1950
1180
903
3970
10083
2526
1702
247
5101
451
374
237
44
183
949
649
739
5180
265
1616
497
2713
95
409
49
702
41
514
664
401
469
251
68
145
1611
1734
1651
705
231
40
245
4245
372
97
497
1867
42
2261
186
158
578
2624
154
47
1675
36
683
3420
230
1172
2645
2905
220
517
234
127
913
1162
68
568
311
454
1771
162
83
283
590
593
404
678
1862
2768
122
486
3441
371
119
673
49
1131
113
2415
675
2504
201
11
155
114
603
1500
968
1153
435
249
1623
709
16
555
365
719
312
278
1745
243
869
104
281
940
258
16
215
676
503
646
861
2808
1096
1404
2215
929
383
340
87
1670
252
852
144
6180
716
2514
1644
6298
11
51
2302
744
803
338
392
13422
420
343
725
365
272
56
16
3921
1420
2562
644
1573
315
75
462
106
8030
60
312
2570
90
980
614
447
1223
41
670
795
317
1922
1762
773
742
786
672
788
513
516
2392
203
240
11
20
35
987
85
866
172
243
393
35
570
1345
316
252
1113
56
469
233
95
1352
694
401
378
404
23
422
2240
218
175
2575
2911
57
599
175
291
24

...

286
2442
1078
2743
844
51
1499
1713
4520
3875
2176
349
1247
2442
17
333
612
49
468
1753
291
454
764
227
151
808
744
450
475
22
853
1314
6056
142
597
279
4980
547
8
1612
29
1924
659
163
1637
275
587
1544
127
263
250
243
1753
156
2475
420
3055
516
2398
348
468
2476
911
316
12057
975
1571
466
214
809
323
187
926
3091
99
266
442
708
1728
321
48
253
457
26
336
513
223
2300
2073
454
1201
2319
791
453
712
1610
133
270
123
611
203
5416
301
57
3074
39
1600
724
475
343
407
552
809
275
277
733
464
13613
1665
100
1881
692
1318
1178
760
688
288
3053
1620
695
250
1541
89
1617
3883
754
1733
153
145
1262
1449
3036
419
2223
1397
303
953
1135
2780
197
714
3635
196
5365
303
526
763
743
1228
352
690
62
328
686
1543
290
387
64
135
816
615
464
1572
444
2612
329
561
187
2688
1785
976
590
307
1599
355
595
2617
142
114
4352
933
591
1547
2442
5200
35
144
176
926
241
2760
1167
454
586
549
953
1578
14
213
3282
86
2451
1342
68
126
857
12133
316
689
2631
470
107
488
6158
1537
387
3173
271
483
230
2589
793
109
1798


23
30
425
95
123
2540
786
582
2751
89
114
444
2787
138
191
298
52
1222
1544
917
1311
38
766
180
3044
341
783
626
4359
1672
373
1260
40
1858
3240
2559
14
699
826
2554
244
1431
3513
879
1347
3802
310
120
329
5976
458
1249
4825
106
2801
2780
3973
571
1554
58
112
715
5384
211
1712
231
39
5512
317
1733
1636
210
500
66
487
465
1134
2475
48
1610
305
278
3197
753
237
240
1354
1175
58
569
929
71
463
1502
3370
2841
53
729
767
982
3089
573
338
63
2950
272
771
5283
2307
240
141
49
322
63
2139
1210
1129
8178
1297
707
965
491
604
12
1109
582
244
245
291
2666
355
1547
95
341
322
3321
4758
1579
1473
5150
718
244
4981
289
100
474
1229
1194
324
2616
418
422
246
62
536
740
1041
685
326
184
2556
101
6366
117
2794
23
703
57
209
157
651
1091
64
339
996
139
1920
4449
439
22
109
4309
626
2001
4084
934
380
1663
13111
481
49
87
1337
653
86
686
2331
951
67
5370
1405
3064
274
1604
1328
196
3010
204
799
6203
489
1818
133
13080
186
3038
4564
2901
10634
1428
2057
167
698
5265
304
97
1252
871
127
1496
625
942
62
1166

323
444
79
2466
1056
1790
85
114
283
381
2533
680
1480
49
1191
715
4534
955
200
70
338
166
788
683
600
482
1129
293
290
12
244
2357
321
1404
5
30
112
265
659
170
365
2064
611
172
847
289
1997
807
1544
245
5617
301
3271
313
430
91
1524
1764
997
11
297
286
782
3209
104
138
42
170
35
325
1982
3870
8449
7008
154
37
1199
3359
1878
705
90
297
72
2829
83
2896
2880
373
167
1949
1455
226
3044
800
1033
175
1241
1585
33
565
5308
1608
277
556
1367
899
5056
1863
378
508
756
142
36
199
674
34
98
2087
214
76
113
2457
1482
866
617
388
106
397
286
695
114
4826
1766
1017
1127
3336
3148
1217
669
10
1181
2910
314
57
189
4260
260
1230
93
545
506
121
1609
491
20
328
1310
270
141
71
370
541
1045
681
242
418
1823
2306
156
747
72
49
150
463
251
33
1540
707
1818
139
263
349
161
2225
192
162
1919
2507
1720
388
56
2613
1614
112
101
853
14
846
689
57
2743
1630
1671
3574
82
465
509
28
541
550
1204
200
200
256
454
1697
96
761
40
188
131
13
662
2116
339
270
159
25
692
76
2214
257
700
903
285
682
503
6347
1357
1540
92

325
128
1789
6298
80
469
1154
34
540
162
2537
268
1277
132
14
249
6
1546
3582
382
11
49
2911
317
2485
1109
41
80
115
180
625
231
31
1602
2329
338
131
942
767
752
10
117
52
451
1678
2008
35
38
1592
476
2760
120
5327
2067
338
2096
388
1485
782
1621
766
506
135
108
2409
595
274
3112
1013
81
66
815
588
1705
360
727
997
925
829
267
424
412
1115
10
1746
122
111
1654
904
1735
2549
952
628
1615
1148
145
96
40
12843
109
673
293
910
49
676
189
210
265
3263
1629
66
520
92
1024
36
1845
50
5422
621
340
829
2492
2491
248
1293
2292
106
5318
1351
385
2373
1479
1156
40
1130
3005
1451
288
541
1225
8567
493
618
48
49
2714
731
1115
1214
2235
2876
396
2782
1818
1340
55
1694
323
710
299
830
2257
12853
859
708
851
6366
823
205
521
2440
61
225
1157
252
864
1679
605
1583
547
233
2777
371
743
67
194
1546
2919
105
442
3099
180
2149
3171
291
426
57
50
752
268
3782
3942
1157
38
172
157
34
2805
2577
661
3275
1781
5102
278
1325
117
398
236
105
1354
2780
1150
3474
960
1270
3744
12121
60
38
140
187
602
535
1487
1833
5

144
2222
397
2076
338
485
2386
169
756
2418
4961
4312
177
2382
3739
637
572
3708
5618
265
520
243
137
1603
337
227
148
2294
2311
98
425
805
303
296
931
53
2861
59
4471
12
42
1404
812
1063
837
578
1530
157
715
13
3698
106
110
180
66
1205
95
540
17
122
61
1716
35
173
1429
1031
52
494
554
2879
269
1404
1285
21
95
1382
332
729
2249
440
727
88
1537
3827
144
2392
338
524
2127
742
2855
402
326
237
796
613
1624
4245
750
41
380
102
1664
14
2664
444
1946
1045
3990
389
363
50
99
2734
366
357
132
480
360
372
3150
487
80
2460
2526
281
579
2776
787
137
1521
5113
137
2589
34
5495
148
160
659
3083
2546
3150
220
1757
1447
660
360
1168
399
303
51
39
40
4945
359
3958
110
511
2413
479
44
3458
1085
3251
73
3083
28
42
2552
201
713
200
242
47
16
121
93
493
1190
895
649
301
2478
513
399
853
453
1115
2328
174
1613
954
712
1384
245
551
12
33
2026
1431
805
2918
43
1900
853
12880
439
473
780
671
259
982
1858
109
382
361
342
100
23
62
2781
3072
169
5268
479
998
281
8
554
3209
180
370
150
925
272
2700
1580
510
985


In [22]:
totalLen/len(files)

1082.9511379570704

In [23]:
totalLen,len(files)

(30120120, 27813)

In [9]:
if __name__=="__main__":
    # test2()
    # test_bug_id()
    project='swt'
    project_names=['swt_']
    # project_names=['swt_','zxing_']
    #已经移到了/data/hdj/cross_project_trans/clen_data
#     code_clean=['/data/hdj/cross_project_trans/clen_data/aspectj/','/data/hdj/cross_project_trans/clen_data/swt/','/data/hdj/cross_project_trans/clen_data/zxing/']
#     code_clean=['/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/clean','/home/hdj/bug_localization/data/SWT/clean/','/home/hdj/bug_localization/data/ZXing/clean/']
#     code_clean_dir=''
    #这些目录已经移到了下/data/hdj/cross_project_trans/report
    report_path='/data/hdj/cross_project_trans/report/bert/'
#     report_path='/data/hdj/cross_project_trans/report/'
#     w2v_file = "/home/hdj/GoogleNews-vectors-negative300.bin"
    #/data/hdj/cross_project_trans/embedding
#     w2v_file = "/home/hdj/cross_project_trans/embedding/word_w2v_100"
    w2v_file='/data/hdj/GoogleNews-vectors-negative300.bin'
    index_num = 1
    # for repeat_index in range(index_num):
    i=0
    for project_name in project_names:
        code_maxl = 100  # code_maxl max statements per file
        code_maxk = 40  # code_maxk max words per statement

        # test_num = 58
        report_maxl = 300  # max report length
#         test_c = 1000  # random choose 300 candidate source code
        code_clean=os.path.join(report_path,project+'_code_clean')
        train_report, train_code, train_labels, W = load_data(code_maxl, code_maxk, report_maxl,w2v_file, report_path,project_name,project_name+'80_',project_name+'20_',code_clean,hdjWord2vec=False)
        i+=1
        # pickle.dump([train_report, train_code, train_labels, W],open("parameters.in", "wb"))
        #pickle.dump([train_report, train_code, train_labels, W],open("JDT_CNN/parameters.in", "wb"))
        print("Finish processing!")

读取80% training_pos的数据
读取80% training_pos的数据完成
读取80% training negative 的数据
train neg:  4dbd09d
读取80% training negative 的数据完成
读取test数据 100
test num : 0
test num : 1
test num : 2
test num : 3
test num : 4
test num : 5
test num : 6
test num : 7
test num : 8
test num : 9
test num : 10
test num : 11
test num : 12
test num : 13
test num : 14
test num : 15
test num : 16
test num : 17
test num : 18
test num : 19
test num : 20
test num : 21
test num : 22
test num : 23
test num : 24
test num : 25
test num : 26
test num : 27
test num : 28
test num : 29
test num : 30
test num : 31
test num : 32
test num : 33
test num : 34
test num : 35
test num : 36
test num : 37
test num : 38
test num : 39
test num : 40
test num : 41
test num : 42
test num : 43
test num : 44
test num : 45
test num : 46
test num : 47
test num : 48
test num : 49
test num : 50
test num : 51
test num : 52
test num : 53
test num : 54
test num : 55
test num : 56
test num : 57
test num : 58
test num : 59
test num : 60
test num : 61
test 

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


max_token :  3000000
load_bin_vec : 3521
<class 'dict'> <class 'collections.defaultdict'>
一共  20558  个词不在词表里
get W  24080 24079
这里显示向量真实大小
24080 300
对train80% 数据进行整理
对train80% 数据进行整理完成
对测试集进行整理
**********
test bug id len 100  test report :  100  test code  100  test label  100  test path  100 test_row_report  100  test_raw_code  100
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 300 300
300 300 300 300 300 3

In [23]:
# source_code_name ='org.aspectj/modules/asm/src/org/aspectj/asm/AsmManager.java'
# # print(os.path.join(pre, source_file_name))
# clean_path='/home/hdj/bug_localization/data/AspectJ/AspectJ-1.5/clean'
# vocab=defaultdict(float)
# project_name='aspectj_'
# test_index=0
# spamreader = csv.reader(open(report_path+project_name+'test/test' + str(test_index) + ".csv", newline=''))
# i=0
# revs_code=[]
# for row in spamreader:
#     if i == 0:
#         i += 1
#         continue
#     label = row[2]
#     source_code_name=row[1]
#     bug_report_id=row[0]
#     with open(os.path.join(clean_path,source_code_name), "r") as f:
#         revs_single = []
#         for line in f:
#             orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
#             if len(orig_rev) <= 1:
#                 orig_rev = "test"
#             words = set(orig_rev.split())
#             for word in words:
#                 vocab[word] += 1
#             if label.strip() == "1":
#                 datum = {"y": 1,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": test_index,
#                          "target": 1,
#                          "path":source_code_name,
#                          "bug_id": bug_report_id}
#             else:
#                 datum = {"y": 0,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": test_index,
#                          "target": 1,
#                          "path": source_code_name,
#                          "bug_id": bug_report_id}#TODO 根据这个区匹配对应的AST
#             revs_single.append(datum)
#         if len(revs_single) == 0:
#             revs_single = revs_pre
#         if len(revs_single)==0:
#             print(os.path.join(clean_path, source_code_name))
#             print('here null :',revs_single,' llllll')
#         revs_code.append(revs_single) #TODO 记录该report对应的code
#         revs_pre = revs_single

In [29]:
# revs_code

In [3]:
# test0=pd.read_pickle('/data/hdj/cross_project_trans/report/aspectj_less_train_data.pkl')
# type(test0)
# test0

In [18]:
w2v=gensim.models.KeyedVectors.load_word2vec_format('/data/hdj/GoogleNews-vectors-negative300.bin', binary=True)
# w2v = Word2Vec.load('/home/hdj/cross_project_trans/embedding/word_w2v_100')#模型的导入

In [6]:
test0=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_test_middle/test0.pkl')
aspectj_more_train_data=pd.read_pickle('/home/hdj/cross_project_trans/report/aspectj_more_train_data.pkl')

In [7]:
aspectj_more_train_data.head()

,bug_id,cnn_report_idx,code_idx,label,raw_report,raw_code,path
0,28974,"[0, 0, 0, 0, 36, 12, 12973, 57, 42, 17, 23, 12...","[[62, 61, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,compiler error introducing field aspect fails ...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
1,28919,"[0, 0, 0, 0, 13030, 284, 289, 295, 273, 276, 2...","[[62, 61, 63, 316, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,waever tries weave methods find exception bug ...,"[{'y': 1, 'text': 'org aspectj weaver bcel', '...",org.aspectj/modules/weaver/src/org/aspectj/wea...
2,29186,"[0, 0, 0, 0, 49, 684, 13106, 253, 13107, 692, ...","[[62, 61, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
3,29186,"[0, 0, 0, 0, 49, 684, 13106, 253, 13107, 692, ...","[[62, 61, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver', 'num_w...",org.aspectj/modules/weaver/src/org/aspectj/wea...
4,29186,"[0, 0, 0, 0, 49, 684, 13106, 253, 13107, 692, ...","[[62, 61, 63, 316, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,ajc emacssym chokes pointcut includes intertyp...,"[{'y': 1, 'text': 'org aspectj weaver bcel', '...",org.aspectj/modules/weaver/src/org/aspectj/wea...


In [27]:
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

In [94]:
#需要为数据的code生成文本形式
def code_merge(raw_code_list):
    code_list=[]
    for raw_code in raw_code_list:
        code_list.append(raw_code['text'])
    code_text=' '.join(code_list)
    return code_text

def data_tensor_generate(data,maxLength=512,ttype='train'):
    '''
        输入是dataframe 包含bug_id	cnn_report_idx	code_idx	label	path	raw_report	raw_code
    '''
    #将raw_code里dict形式合并生成code_text
    data['raw_code_text']=data['raw_code'].apply(code_merge)
    #生成完成
    all_labels=list(data['label'])
    all_cnn_report_idx=list(data['cnn_report_idx'])
    all_code_idx=list(data['code_idx'])
    raw_report=list(data['raw_report'])
    raw_code=list(data['raw_code_text'])
    reportInputs=tokenizer.batch_encode_plus(raw_report,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
    codeInputs=tokenizer.batch_encode_plus(raw_code,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
    all_report_input_ids=reportInputs['input_ids']
    all_report_input_mask=reportInputs['attention_mask']
    all_code_input_ids=codeInputs['input_ids']
    all_code_input_mask=codeInputs['attention_mask']
    
    all_report_input_ids = torch.tensor(all_report_input_ids, dtype=torch.long)
    all_report_input_mask = torch.tensor(all_report_input_mask, dtype=torch.long)
    all_code_input_ids = torch.tensor(all_code_input_ids, dtype=torch.long)
    all_code_input_mask = torch.tensor(all_code_input_mask, dtype=torch.long)
    
    all_cnn_report_idx=torch.tensor(all_cnn_report_idx, dtype=torch.long)
    all_code_idx=torch.tensor(all_code_idx, dtype=torch.long)
    all_labels = torch.tensor(all_labels, dtype=torch.long)

    dataset = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels)
    if ttype=='train':
        return dataset
    else:
        return dataset,data
dataset=data_tensor_generate(aspectj_more_train_data)

    #
# raw_report[:3]

In [95]:
dataset.tensors[0].shape,dataset.tensors[1].shape,dataset.tensors[2].shape,dataset.tensors[3].shape,dataset.tensors[4].shape,dataset.tensors[5].shape,dataset.tensors[6].shape

(torch.Size([1668, 512]),
 torch.Size([1668, 512]),
 torch.Size([1668, 512]),
 torch.Size([1668, 512]),
 torch.Size([1668, 208]),
 torch.Size([1668, 100, 40]),
 torch.Size([1668]))

In [97]:
aspectj_w= pickle.load(open('/home/hdj/cross_project_trans/report/aspectj_test_middle/w_parameters.in','rb'))

In [101]:
aspectj_w[0].shape

(29864, 100)

In [102]:
all_cnn_report_idx=dataset.tensors[4]
all_code_idx=dataset.tensors[5]
aspectj_w= pickle.load(open('/home/hdj/cross_project_trans/report/aspectj_test_middle/w_parameters.in','rb'))
embedding = nn.Embedding.from_pretrained(torch.tensor(aspectj_w[0]))#从已有的weight导入embedding
train_code = embedding(all_cnn_report_idx)
train_report =embedding(all_code_idx)

In [103]:
train_code.shape,train_report.shape

(torch.Size([1668, 208, 100]), torch.Size([1668, 100, 40, 100]))

In [84]:
# report=['a a a a a a b b b b','a a a a a a b b b b']
report=raw_report
inputs=tokenizer.batch_encode_plus(report,add_special_tokens=True,max_length=10,pad_to_max_length=True,return_token_type_ids=True,truncation=True)

In [86]:
# inputs

In [59]:
tokenizer.convert_tokens_to_ids('a'),tokenizer.convert_ids_to_tokens(10),tokenizer.convert_ids_to_tokens(0),tokenizer.convert_ids_to_tokens(2)

(102, 'Ġa', '<s>', '</s>')

In [62]:
tokenizer.convert_ids_to_tokens(1)

'<pad>'

In [96]:
len(aspectj_more_train_data['raw_code_text'])

1668

In [68]:
len(aspectj_more_train_data['raw_code'][0])

for index, row in aspectj_more_train_data.iterrows():
    all_code_idx.append(row['code_idx'])
    all_cnn_report_idx.append(row['cnn_report_idx'])
    all_labels.append(row['label'])
    #处理report编码
    reportInputs=tokenizer.encode_plus(row['raw_report'],add_special_tokens=True,max_length=512,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
    all_report_input_ids.append(reportInputs['input_ids'])
    all_report_input_mask.append(reportInputs['attention_mask'])
    #处理report编码完成
    #处理code编码
    code_list=[]
    for raw_code in row['raw_code']:
        code_list.append(raw_code['text'])
    code_text=' '.join(code_list)
    codeInputs=tokenizer.encode_plus(code_text,add_special_tokens=True,max_length=10,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
    all_code_input_ids.append(codeInputs['input_ids'])
    all_code_input_mask.append(codeInputs['attention_mask'])
    #处理code编码完成
#     print(type(aspectj_more_train_data),row['label'],row['cnn_report_idx'])
#     break

In [67]:
print(all_report_input_ids)
print(all_report_input_mask)
print(all_code_input_ids)
print(all_code_input_mask)
print(all_labels)
print(all_cnn_report_idx)
print(all_code_idx)

[[0, 44904, 5849, 10345, 882, 6659, 10578, 33172, 5591, 2]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[[0, 31118, 6659, 267, 52, 9903, 31118, 6659, 267, 2]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
[1]
[[0, 0, 0, 0, 36, 12, 12973, 57, 42, 17, 23, 12972, 50, 12, 49, 55, 28, 22, 43, 55, 28, 22, 47, 43, 29, 44, 11, 12986, 39, 12977, 55, 28, 22, 12976, 43, 8, 46, 41, 57, 55, 28, 22, 19, 49, 13, 58, 25, 31, 48, 37, 12, 55, 28, 22, 55, 28, 22, 47, 12978, 18, 52, 3, 9, 20, 56, 12985, 51, 12980, 30, 27, 43, 12982, 38, 12987, 59, 53, 32, 4, 30, 27, 43, 24, 12984, 42, 12976, 45, 40, 19, 12978, 14, 7, 12975, 19, 25, 21, 19, 12983, 19, 12979, 25, 2, 26, 12979, 1, 15, 6, 53, 16, 54, 12974, 35, 12981, 33, 34, 60, 47, 34, 60, 6, 57, 28, 22, 5, 60, 34, 60, 47, 34, 60, 5, 60, 34, 60, 34, 60, 34, 60, 47, 10, 47, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[arra

In [3]:
test0.head()

,bug_id,cnn_report_idx,code_idx,label,path,raw_report,raw_code
0,146546,"[0, 0, 0, 0, 1937, 4128, 19718, 4679, 19720, 2...","[[62, 61, 828, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1,org.aspectj/modules/asm/src/org/aspectj/asm/As...,remove hard coded dependency getfilename metho...,"[{'y': 1, 'text': 'org aspectj asm', 'num_word..."
1,146546,"[0, 0, 0, 0, 1937, 4128, 19718, 4679, 19720, 2...","[[62, 61, 828, 87, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,org.aspectj/modules/asm/src/org/aspectj/asm/in...,remove hard coded dependency getfilename metho...,"[{'y': 1, 'text': 'org aspectj asm intern', 'n..."
2,146546,"[0, 0, 0, 0, 1937, 4128, 19718, 4679, 19720, 2...","[[309, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,org.aspectj/modules/tests/base/test135/Driver....,remove hard coded dependency getfilename metho...,"[{'y': 0, 'text': 'test', 'num_words': 1, 'tes..."
3,146546,"[0, 0, 0, 0, 1937, 4128, 19718, 4679, 19720, 2...","[[62, 61, 309, 997, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0,org.aspectj/modules/aspectj-attic/testing-src/...,remove hard coded dependency getfilename metho...,"[{'y': 0, 'text': 'org aspectj test compar', '..."
4,146546,"[0, 0, 0, 0, 1937, 4128, 19718, 4679, 19720, 2...","[[43, 325, 706, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,org.aspectj/modules/tests/bugs150/pr113947/cas...,remove hard coded dependency getfilename metho...,"[{'y': 0, 'text': 'java util collect', 'num_wo..."


In [11]:
test0['raw_report'][0]

'remove hard coded dependency getfilename methods get file name part bug need remove hard coded dependency methods use handles get filename methods asmmanager getfilename string aspectjelementhierarchy getfilename string asm manager get file name aspect element hierarchy get file name'

In [15]:
X = pickle.load(open('/home/hdj/NPCNN_hdj/parameters.in','rb'))

In [122]:
# X[0]
import gensim

In [123]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('/data/hdj/GoogleNews-vectors-negative300.bin', binary=True)

In [106]:
vocab_emd=w2v.wv.vocab

In [129]:
w2v['author'].shape

(300,)

In [113]:
aspectj_data=pickle.load(open('/home/hdj/NPCNN_hdj/parameters.in','rb'))

In [115]:
w=aspectj_data[3]
len(w[0])

300

In [61]:
# def load_data( code_maxl, code_maxk, report_maxl, w2v_file, report_path,project_name,more_project_name,less_project_name,clean_path,hdjWord2vec):
#     '''
#         code_maxl:代码的最大行数 100
#         code_maxk:代码每行最大单词数 40 
#         report_maxl:report最大的单词数 308
#         w2c_file: 词向量所在位置/data/hdj/cross_project_trans/embedding/word_w2v_100
#         report_path:/data/hdj/cross_project_trans/report/bert 文件保存所在位置
#         project_name:将要处理的项目aspectj_
#         more_project_name:aspectj_80_
#         less_project_name:aspectj_20_
#         clean_path:清洗过后的项目路径/data/hdj/cross_project_trans/report/bert/swt_code_clean
        
#     '''
#     less_report_ids=[]
#     #revs:保存report的dict信息
#     '''
#     #revs_code:保存一个code文件的所有行 [
#                             [{"y": 1,#这是一行的数据
#                              "text": orig_rev,
#                              "num_words": len(orig_rev.split()),
#                              "test": -1,
#                              "target": -1,
#                              "path": source_code_name,
#                              'bug_id': bug_report_id},{},{}这是很多行
#                          ]
#                         ]
#     '''
    
#     report_ids,revs, revs_code = [], [],[]
#     report, code, train_path = [], [], []
#     revs_pre = []
#     vocab = defaultdict(float)
#     #拼接成的路径 /data/hdj/cross_project_trans/report/aspectj_report_clean.csv
#     #/data/hdj/cross_project_trans/report/bert/swt_report_clean.csv
#     report_data=pd.read_csv(report_path+project_name+'report_clean.csv')
#     print('读取80% training_pos的数据')#读取80% training_pos的数据/data/hdj/cross_project_trans/report/aspectj_80_training_pos.csv
#     spamreader = csv.reader(open(report_path+more_project_name+"training_pos.csv", newline=''))
#     i=0
#     for row in spamreader:
#         if i==0:
#             i+=1
#             continue
#         #根据bugid获得对应report
#         bug_report_id = row[0].strip()
#         report_ids.append(bug_report_id)#TODO 记录report_id
#         bug_description = getReport(bug_report_id,report_data, report_maxl,'ll')
#         # report信息获得完成
#         # 统计词表信息
#         orig_rev = bug_description.strip().lower()
#         words = set(orig_rev.split())
#         for word in words:
#             vocab[word] += 1
#         #统计完成
#         #将report相关信息保存
#         datum = {"y": 1,#是正例
#                  "text": orig_rev,#report内容
#                  "num_words": len(orig_rev.split()),#report长度
#                  "test": -1,#不是测试数据
#                  "target": -1,
#                  'bug_id': bug_report_id}
#         revs.append(datum)#TODO 记录report
#         #保存完成

#         source_code_name = row[1].strip()
#         #clean_path+source_code_name='/home/hdj/bug_localization/data/ZXing/clean/' + 'com/google/zxing/client/rim/util/Log.java'
#         #/data/hdj/cross_project_trans/report/bert/jdt_code
#         with open(os.path.join(clean_path,source_code_name), "r") as f:
#             revs_single = []#保存一个java文件所有行[[import pandas as pd],[],[]]
#             for k,line in enumerate(f):
#                 orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
#                 if len(orig_rev) <= 1: continue#这里过滤了数量小于1的行
#                 words = set(orig_rev.split())
#                 for word in words:
#                     vocab[word] += 1
#                 datum = {"y": 1,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": -1,
#                          "target": -1,
#                          "path": source_code_name,
#                          'bug_id': bug_report_id}
#                 revs_single.append(datum)
#                 if len(revs_single)>=code_maxl:
#                     break
#             revs_code.append(revs_single)#保存所有的java文件内容 [[revs_single],[]] #TODO 记录该report对应的code
#     # 读取80% training_pos的数据完成
#     print('读取80% training_pos的数据完成')

# #     print('读取20% training_pos的数据')
# #     # 读取20% training_pos的数据
# #     spamreader = csv.reader(open(report_path + less_project_name + "training_pos.csv", newline=''))
# #     i = 0
# #     for row in spamreader:
# #         if i == 0:
# #             i += 1
# #             continue
# #         # 根据bugid获得对应report
# #         bug_report_id = row[0].strip()
# #         less_report_ids.append(bug_report_id)  # TODO 记录report_id
# #         bug_description = getReport(bug_report_id, report_data, report_maxl, 'll')
# #         # report信息获得完成
# #         # 统计词表信息
# #         orig_rev = bug_description.strip().lower()
# #         words = set(orig_rev.split())
# #         for word in words:
# #             vocab[word] += 1
# #         # 统计完成
# #         # 将report相关信息保存
# #         datum = {"y": 1,  # 是正例
# #                  "text": orig_rev,  # report内容
# #                  "num_words": len(orig_rev.split()),  # report长度
# #                  "test": -2,  # -1 代表80%的数据   -2 代表20%的数据
# #                  "target": -1,
# #                  'bug_id': bug_report_id}
# #         revs.append(datum)  # TODO 记录report
# #         # 保存完成

# #         source_code_name = row[1].strip()
# #         # clean_path+source_code_name='/home/hdj/bug_localization/data/ZXing/clean/' + 'com/google/zxing/client/rim/util/Log.java'
# #         with open(os.path.join(clean_path, source_code_name), "r") as f:
# #             revs_single = []  # 保存一个java文件所有行[[import pandas as pd],[],[]]
# #             for line in f:
# #                 orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
# #                 if len(orig_rev) <= 1: continue
# #                 words = set(orig_rev.split())
# #                 for word in words:
# #                     vocab[word] += 1
# #                 datum = {"y": 1,
# #                          "text": orig_rev,
# #                          "num_words": len(orig_rev.split()),
# #                          "test": -2,
# #                          "target": -1,
# #                          "path": source_code_name,
# #                          'bug_id': bug_report_id}
# #                 revs_single.append(datum)

# #             revs_code.append(revs_single)  # 保存所有的java文件内容 [[revs_single],[]] #TODO 记录该report对应的code
# #     # 读取20% training_pos的数据完成
# #     print('读取20% training_pos的数据完成')

#     print('读取80% training negative 的数据')
#     #读取80% training negative 的数据
#     spamreader = csv.reader(open(report_path+more_project_name+"training_neg.csv", newline=''))
#     i=0
#     for time,row in enumerate(spamreader):
#         if i==0:
#             i+=1
#             continue
#        # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
        
#         bug_report_id = row[0].strip()
#         if time%10000==0:
#             print('train neg: ',bug_report_id)
#         report_ids.append(bug_report_id)#TODO 记录report_id

#         #统计词频
#         bug_description = getReport(bug_report_id, report_data, report_maxl,'aa')
#         # rev = []
#         # rev.append(bug_description.strip())
#         # orig_rev = " ".join(rev).lower()
#         orig_rev = bug_description.strip().lower()
#         words = set(orig_rev.split())
#         for word in words:
#             vocab[word] += 1
#         #统计完成


#         datum = {"y": 0,
#                  "text": orig_rev,
#                  "num_words": len(orig_rev.split()),
#                  "test": -1,
#                  "target": -1,
#                  "bug_id":bug_report_id}
#         revs.append(datum)#TODO 记录report

#         source_code_name = row[1].strip()
#         with open(os.path.join(clean_path,source_code_name), "r") as f:
#             revs_single = []
#             for k,line in enumerate(f):
#                 orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
#                 if len(orig_rev) <= 1: continue
#                 words = set(orig_rev.split())
#                 for word in words:
#                     vocab[word] += 1
#                 datum = {"y": 0,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": -1,
#                          "target": -1,
#                         "path":source_code_name,
#                          "bug_id": bug_report_id}
#                 revs_single.append(datum)
#                 if len(revs_single)>=code_maxl:
#                     break
#             if len(revs_single)==0:#TODO 注意 过滤后的java文件不应该为空，否则后续处理有问题
#                 print('this is the bug')
#             revs_code.append(revs_single)#TODO 记录该report对应的code
#     # 读取80% training negative 的数据完成
#     print('读取80% training negative 的数据完成')

# #     print('读取20% training negative 的数据')
# #     # 读取20% training negative 的数据
# #     spamreader = csv.reader(open(report_path + less_project_name + "training_neg.csv", newline=''))
# #     i = 0
# #     for row in spamreader:
# #         if i == 0:
# #             i += 1
# #             continue
# #             # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
# #         bug_report_id = row[0].strip()
# #         less_report_ids.append(bug_report_id)  # TODO 记录report_id
# #         # 统计词频
# #         bug_description = getReport(bug_report_id, report_data, report_maxl, 'aa')
# #         # rev = []
# #         # rev.append(bug_description.strip())
# #         # orig_rev = " ".join(rev).lower()
# #         orig_rev = bug_description.strip().lower()
# #         words = set(orig_rev.split())
# #         for word in words:
# #             vocab[word] += 1
# #         # 统计完成
# #         datum = {"y": 0,
# #                  "text": orig_rev,
# #                  "num_words": len(orig_rev.split()),
# #                  "test": -2,
# #                  "target": -1,
# #                  "bug_id": bug_report_id}
# #         revs.append(datum)  # TODO 记录report
# #         source_code_name = row[1].strip()
# #         with open(os.path.join(clean_path, source_code_name), "r") as f:
# #             revs_single = []
# #             for line in f:
# #                 orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
# #                 if len(orig_rev) <= 1: continue
# #                 words = set(orig_rev.split())
# #                 for word in words:
# #                     vocab[word] += 1
# #                 datum = {"y": 0,
# #                          "text": orig_rev,
# #                          "num_words": len(orig_rev.split()),
# #                          "test": -2,
# #                          "target": -1,
# #                          "path": source_code_name,
# #                          "bug_id": bug_report_id}
# #                 revs_single.append(datum)
# #             if len(revs_single) == 0:  # TODO 注意 过滤后的java文件不应该为空，否则后续处理有问题
# #                 print('this is the bug')
# #             revs_code.append(revs_single)  # TODO 记录该report对应的code
# #     # 读取20% training negative 的数据完成
# #     print('读取20% training negative 的数据完成')
#     # 读取test数据  test_num ：number of test bug reports #TODO 数量不固定，提前传个参数
#     #test_path=report_path+project_name+'test/test'+str(index)+'.csv'
#     #  例如    '/home/hdj/cross_project_trans/report/'+'aspectj_'+'test/test'+'0'+'.csv'
#     test_path=report_path+project_name+'test/'
# #     test_num=len(os.listdir(test_path))
#     test_num=100
#     print('读取test数据',test_num)
#     for test_index in range(test_num):
#         print('test num :',test_index)
#         spamreader = csv.reader(open(report_path+project_name+'test/test' + str(test_index) + ".csv", newline=''))
#         i=0
#         for row in spamreader:
#             if i == 0:
#                 i += 1
#                 continue
#             label = row[2]
#             # bug_report_name = "JDT_CNN/BugCorpus/BugCorpus/" + row[0].strip()
#             bug_report_id = row[0].strip()
#             # report_ids.append(bug_report_id)#TODO 记录test 的report_id

#             bug_description = getReport(bug_report_id, report_data, report_maxl,'dd')
#             orig_rev = bug_description.strip().lower()
#             words = set(orig_rev.split())
#             for word in words:
#                 vocab[word] += 1
#             if label.strip() == "1":
#                 datum = {"y": 1,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": test_index,#用来记录test索引 每个测试report都单独处理
#                          "target": 1,
#                          "bug_id": bug_report_id}
#             else:
#                 datum = {"y": 0,
#                          "text": orig_rev,
#                          "num_words": len(orig_rev.split()),
#                          "test": test_index,
#                          "target": 1,
#                          "bug_id": bug_report_id}
#             #revs 存放report信息
#             revs.append(datum)#TODO 记录test report

#           #  source_file_name = "JDT_CNN/sources/sources/JDT_4_5/" + row[1].strip()
#             source_code_name = row[1].strip()
#             # print(os.path.join(pre, source_file_name))
#             with open(os.path.join(clean_path,source_code_name), "r") as f:
#                 revs_single = []
#                 for k,line in enumerate(f):
#                     orig_rev = clean_str(str(line.strip()))  # TODO 更改代码的正则化
#                     if len(orig_rev) <= 1:
#                         orig_rev = "test"
#                     words = set(orig_rev.split())
#                     for word in words:
#                         vocab[word] += 1
#                     if label.strip() == "1":
#                         datum = {"y": 1,
#                                  "text": orig_rev,
#                                  "num_words": len(orig_rev.split()),
#                                  "test": test_index,
#                                  "target": 1,
#                                  "path":source_code_name,
#                                  "bug_id": bug_report_id}
#                     else:
#                         datum = {"y": 0,
#                                  "text": orig_rev,
#                                  "num_words": len(orig_rev.split()),
#                                  "test": test_index,
#                                  "target": 1,
#                                  "path": source_code_name,
#                                  "bug_id": bug_report_id}#TODO 根据这个区匹配对应的AST
#                     revs_single.append(datum)
#                     if len(revs_single)>=code_maxl:
#                         break
#                 if len(revs_single) == 0:
#                     revs_single = revs_pre
#                 if len(revs_single)==0:
#                     print(os.path.join(clean_path, source_code_name))
#                     print('here null :',revs_single,' llllll')
#                 revs_code.append(revs_single) #TODO 记录该report对应的code
#                 revs_pre = revs_single
#     print('读取test数据 ', test_num,' 完成')
#     w2v = load_bin_vec('',w2v_file, vocab,hdjWord2vec)
#     if hdjWord2vec:
#         add_unknown_words(w2v, vocab,k=100)
#         W, word_idx_map = get_W(w2v,k=100)
#     else:
#         add_unknown_words(w2v, vocab)
#         W, word_idx_map = get_W(w2v)
#     print('这里显示向量真实大小')
#     print(len(W),len(W[0]))
#     #对train80% 数据进行整理
#     print('对train80% 数据进行整理')
#     train_id,train_report, train_code, train_labels = [], [], [],[]
#     #1. 对report信息进行操作
#     for rev in revs:
        
#         if rev["test"] == -1:
#             sent = getIdxfrom_sent_n(rev["text"], report_maxl, word_idx_map, filter_h=5)
#             train_id.append(rev["bug_id"])
#             train_report.append(sent)
#             report.append(rev['text'])
#     #2. 对code信息进行操作
#     for revs_single in revs_code:
#         data_single = []
#         label_single = []
#         if len(revs_single)==0:
#             print('null ',revs_single,' kl')
#         for rev in revs_single:
#                 sent = getIdxfrom_sent(rev["text"], word_idx_map, code_maxk=code_maxk)
#                 data_single.append(sent)
#         if rev["test"] == -1:
            
#             data_single = np.array(data_single, dtype="int")
#             if len(data_single)==0:
#                 print(type(data_single),data_single,revs_single)
# #             try:
#             data_single = alignData(data_single, code_maxl,code_maxk)
#             train_code.append(data_single)
#             label_single.append(rev["y"])
#             train_labels.append(label_single)
#             code.append(revs_single)
#             train_path.append(rev['path'])
# #             except:
# #                 continue

#     train_labels = np.array(train_labels, dtype="int")
#     #train80% 数据整理完成
#     print('对train80% 数据进行整理完成')

#     #对测试集进行整理
#     print('对测试集进行整理')
#     test_report, test_code, test_path, test_bug_id, test_raw_code, test_row_report = [], [], [], [], [], []
#     test_labels = []
#     #TODO 这里test_num也需要根据项目来指定
#     for i in range(test_num):
#         test_report.append([])
#         test_code.append([])
#         test_labels.append([])
#         test_path.append([])
#         test_bug_id.append([])
#         test_raw_code.append([])
#         test_row_report.append([])
#     #1. report进行整理
#     for rev in revs:
#         # sent = []
#         index = rev["test"]
#         if index >= 0:
#             sent = getIdxfrom_sent_n(rev["text"], report_maxl, word_idx_map, filter_h=5)
#             test_bug_id[index].append(rev['bug_id'])
#             test_report[index].append(sent)
#             test_row_report[index].append(rev['text'])
#     #report 整理完成

#     #2. 对test code进行整理
#     for revs_single in revs_code:
#         data_single = []
#         label_single = []
#         for rev in revs_single:
#             sent = getIdxfrom_sent(rev["text"], word_idx_map, code_maxk=code_maxk)
#             data_single.append(sent)

#         index = rev["test"]
#         if index >= 0:
#             label_single.append(rev["y"])
#             data_single = np.array(data_single, dtype="int")
#             data_single = alignData(data_single, code_maxl,code_maxk)
#             test_code[index].append(data_single)
#             test_labels[index].append(label_single)
#             test_path[index].append(rev['path'])
#             test_raw_code[index].append(revs_single)
# #     for revs_single in revs_code:
# #         index = rev["test"]
# #         if index >= 0:
# #             data_single = []
# #             label_single = []
# #             for rev in revs_single:
# #                 sent = getIdxfrom_sent(rev["text"], word_idx_map, code_maxk=code_maxk)
# #                 data_single.append(sent)
# #             label_single.append(rev["y"])
# #             data_single = np.array(data_single, dtype="int")
# #             data_single = alignData(data_single, code_maxl)
# #             test_code[index].append(data_single)
# #             test_labels[index].append(label_single)
# #             test_path[index].append(rev['path'])
# #             test_raw_code[index].append(revs_single)
#     #test code整理完成

#     #single_path='singleProject/'
#     single_path=''
#     #3. 将test数据保存
#     print('*' * 10)
#     print('test bug id len', len(test_bug_id), ' test report : ', len(test_report), ' test code ', len(test_code),
#           ' test label ', len(test_labels), ' test path ', len(test_path), 'test_row_report ', len(test_row_report)
#           , ' test_raw_code ', len(test_raw_code))
# #     ast_path='/data/hdj/cross_project_trans/out/'+project_name+'tree_blocks.pkl'
# #     ast_file = pd.read_pickle(ast_path)
# #     ast_file = ast_file[['path', 'ast_code']]#TODO 将ast生成时字段改为ast_code
#     for i in range(test_num):
#         test_labels[i] = np.array(test_labels[i], dtype="int")
#         test_data = pd.DataFrame()
#         #300 300 0 0 0 300 0
#         print(len(test_bug_id[i]),len(test_report[i]),len(test_code[i]),len(test_labels[i]),len(test_path[i]),len(test_row_report[i]),len(test_raw_code[i]))
#         test_data['bug_id'] = test_bug_id[i]
#         test_data['cnn_report_idx'] = test_report[i]
#         test_data['code_idx'] = test_code[i]
#         test_data['label'] = test_labels[i]
#         test_data['path'] = test_path[i]
#         test_data['raw_report'] = test_row_report[i]
#         test_data['raw_code'] = test_raw_code[i]
#         # 合并ast数据
# #         test_data = pd.merge(test_data, ast_file, on='path', how='left')
#         test_data.to_pickle(report_path+single_path+project_name+'test_middle/test' + str(i) + '.pkl')
#         # print('bug_id ', test_bug_id[i])
#         # print('test_report ', test_report[i])
#         # print('test_code ', test_code[i])
#         # print('test_labels ', test_labels[i])
#         # print('test_path ', test_path[i])
#         # print('test_row_report ', test_row_report[i])
#         # print('test_raw_code ', test_raw_code[i])
#     #test数据保存完成
#     #测试集整理完成

#     #    random permutation; print to parameters.in
#     # train_report, train_code, train_labels = random_permutation(train_report, train_code, train_labels)

#     #80% train数据保存
#     print('vaocab ', len(vocab))
#     print('word_idx_map : ', len(word_idx_map))
#     # print(report_ids[0])
#     # print(train_report[0])
#     # print(train_code[0])
#     # print(train_labels[0])
#     # print(report[0])
#     # print(code[0])
#     # print(train_path[0])
#     train_data = pd.DataFrame()
#     print('数据长度 : ', len(report_ids), len(train_report), len(train_code), len(train_labels), len(report), len(code),
#           len(train_path))
#     train_data['bug_id'] = report_ids
#     train_data['cnn_report_idx'] = train_report
#     train_data['code_idx'] = train_code
#     train_data['label'] = train_labels
#     train_data['raw_report'] = report
#     train_data['raw_code'] = code
#     train_data['path'] = train_path
# #     train_data = pd.merge(train_data, ast_file, on='path', how='left')
#     train_data.to_pickle(report_path+single_path+project_name+'more_train_data.pkl')
#     #80% 数据保存完成
#     #20%数据保存
#     # print(less_report_ids[0])
#     # print(less_train_report[0])
#     # print(less_train_code[0])
#     # print(less_train_labels[0])
#     # print(less_report[0])
#     # print(less_code[0])
#     # print(less_train_path[0])
# #     less_train_data = pd.DataFrame()
# #     print('数据长度 : ', len(less_report_ids), len(less_train_report), len(less_train_code), len(less_train_labels), len(less_report), len(less_code),
# #           len(less_train_path))
# #     less_train_data['bug_id'] = less_report_ids
# #     less_train_data['cnn_report_idx'] = less_train_report
# #     less_train_data['code_idx'] = less_train_code
# #     less_train_data['label'] = less_train_labels
# #     less_train_data['raw_report'] = less_report
# #     less_train_data['raw_code'] = less_code
# #     less_train_data['path'] = less_train_path
# # #     less_train_data = pd.merge(less_train_data, ast_file, on='path', how='left')
# #     less_train_data.to_pickle(report_path +single_path+ project_name + 'less_train_data.pkl')
#     #20%数据保存完成
#     print("Finish loading")

#     pickle.dump([W], open(report_path+single_path + project_name + 'test_middle/'+"w_parameters.in", "wb"))
#     return train_report, train_code, train_labels, W